<a href="https://colab.research.google.com/github/Sashimimeow/BSC_DPDM23/blob/main/G_project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from matplotlib import pyplot as plt
from sklearn.metrics import accuracy_score

!pip install apyori
from apyori import apriori

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn import tree

In [23]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Import Data

In [24]:
data_truck = pd.read_excel('/content/drive/MyDrive/bsc_dpdm_23/Final_Project/ข้อมูลอุบัติเหตุรถบรรทุก (5).xlsx')

## Data truck 2560

In [25]:
data_dict_truck = pd.read_excel('/content/drive/MyDrive/bsc_dpdm_23/Final_Project/ข้อมูลอุบัติเหตุรถบรรทุก (5).xlsx', sheet_name = 0)
data_dict_truck.head()

,Column,Description,Data Type,ชื่อตัวแปร,ค่าที่เป็นไปได้
0,Date,วันเดือนปีที่เกิดเหตุ,number,NaN,NaN
1,Time,ช่วงเวลาที่เกิดเหตุ,number,16.01 - 20.00 น.,1.0
2,NaN,NaN,NaN,20.01 - 00.00 น.,2.0
3,NaN,NaN,NaN,00.01 - 04.00 น.,3.0
4,NaN,NaN,NaN,04.01 - 08.00 น.,4.0


In [26]:
data_2560 = pd.read_excel('/content/drive/MyDrive/bsc_dpdm_23/Final_Project/ข้อมูลอุบัติเหตุรถบรรทุก (5).xlsx', sheet_name = 1)
data_2560.head()

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,Date,Time,Province,Loss,Party,Cause,Main cause of accidents,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16
0,02/01/2560,11:16:00,ขอนแก่น,3,รถอื่นๆ,รถบรรทุก,หลับใน,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,04/01/2560,07:30:00,แพร่,2,ไม่มีคู่กรณี,รถบรรทุก,หลับใน,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,04/01/2560,20:10:00,สระบุรี,2,รถยนต์นั่งส่วนบุคคลเกิน7คน,รถยนต์นั่งส่วนบุคคลเกิน7คน,ขับแซงอย่างผิดกฎหมาย/แซงในที่คับขัน,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,04/01/2560,21:10:00,เพชรบุรี,2,รถจักรยานยนต์,รถจักรยานยนต์,ขับแซงอย่างผิดกฎหมาย/แซงในที่คับขัน,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,05/01/2560,07:30:00,เชียงราย,2,รถยนต์นั่งส่วนบุคคลเกิน7คน,รถยนต์นั่งส่วนบุคคลเกิน7คน,ขับรถเร็ว,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
data60_col = data_2560[['Date','Time','Province','Loss','Party','Cause','Main cause of accidents']]
data60_col.head()  #เลือกเฉพาะคอลัมน์ที่ต้องการ

,Date,Time,Province,Loss,Party,Cause,Main cause of accidents
0,02/01/2560,11:16:00,ขอนแก่น,3,รถอื่นๆ,รถบรรทุก,หลับใน
1,04/01/2560,07:30:00,แพร่,2,ไม่มีคู่กรณี,รถบรรทุก,หลับใน
2,04/01/2560,20:10:00,สระบุรี,2,รถยนต์นั่งส่วนบุคคลเกิน7คน,รถยนต์นั่งส่วนบุคคลเกิน7คน,ขับแซงอย่างผิดกฎหมาย/แซงในที่คับขัน
3,04/01/2560,21:10:00,เพชรบุรี,2,รถจักรยานยนต์,รถจักรยานยนต์,ขับแซงอย่างผิดกฎหมาย/แซงในที่คับขัน
4,05/01/2560,07:30:00,เชียงราย,2,รถยนต์นั่งส่วนบุคคลเกิน7คน,รถยนต์นั่งส่วนบุคคลเกิน7คน,ขับรถเร็ว


In [28]:
#เปลี่ยนชื่อจังหวัดเป็นภาษาอังกฤษ
data_truck_60 = data60_col
datamain60 = pd.DataFrame(data_truck_60)
datamain60['Province'] = datamain60['Province'].apply(lambda x: x.strip())
datamain60['Province'] = datamain60['Province'].replace({
    'เชียงใหม่': 'Chiang Mai',
    'เชียงราย': 'Chiang Rai',
    'ลำปาง': 'Lampang',
    'ลำพูน': 'Lamphun',
    'แม่ฮ่องสอน': 'Mae Hong Son',
    'น่าน': 'Nan',
    'พะเยา': 'Phayao',
    'แพร่': 'Phrae',
    'อุตรดิตถ์': 'Uttaradit',
    'กาฬสินธุ์': 'Kalasin',
    'ขอนแก่น': 'Khon Kaen',
    'ชัยภูมิ': 'Chaiyaphum',
    'นครพนม': 'Nakhon Phanom',
    'นครราชสีมา': 'Nakhon Ratchasima',
    'บึงกาฬ': 'Bueng Kan',
    'บุรีรัมย์': 'Buriram',
    'มหาสารคาม': 'Maha Sarakham',
    'มุกดาหาร': 'Mukdahan',
    'ยโสธร': 'Yasothon',
    'ร้อยเอ็ด': 'Roi Et',
    'เลย': 'Loei',
    'สกลนคร': 'Sakon Nakhon',
    'สุรินทร์': 'Surin',
    'ศรีสะเกษ': 'Sisaket',
    'หนองคาย': 'Nong Khai',
    'หนองบัวลำภู': 'Nong Bua Lamphu',
    'อุดรธานี': 'Udon Thani',
    'อุบลราชธานี': 'Ubon Ratchathani',
    'อำนาจเจริญ': 'Amnat Charoen',
    'กทม.': 'Bangkok',
    'กทม': 'Bangkok',
    'กรุงเทพมหานคร': 'Bangkok',
    'กำแพงเพชร': 'Kamphaeng Phet',
    'ชัยนาท': 'Chai Nat',
    'นครนายก': 'Nakhon Nayok',
    'นครปฐม': 'Nakhon Pathom',
    'นครสวรรค์': 'Nakhon Sawan',
    'นนทบุรี': 'Nonthaburi',
    'ปทุมธานี': 'Pathum Thani',
    'พระนครศรีอยุธยา': 'Phra Nakhon Si Ayutthaya',
    'พระนครศรี อยุธยา': 'Phra Nakhon Si Ayutthaya',
    'พิจิตร': 'Phichit',
    'พิษณุโลก': 'Phitsanulok',
    'เพชรบูรณ์': 'Phetchabun',
    'ลพบุรี': 'Lopburi',
    'สมุทรปราการ': 'Samut Prakan',
    'สมุทรสงคราม': 'Samut Songkhram',
    'สมุทรสาคร': 'Samut Sakhon',
    'สิงห์บุรี': 'Sing Buri',
    'สุโขทัย': 'Sukhothai',
    'สุพรรณบุรี': 'Suphan Buri',
    'สระบุรี': 'Saraburi',
    'อ่างทอง': 'Ang Thong',
    'อุทัยธานี': 'Uthai Thani',
    'จันทบุรี': 'Chanthaburi',
    'ฉะเชิงเทรา': 'Chachoengsao',
    'ชลบุรี': 'Chonburi',
    'ตราด': 'Trat',
    'ปราจีนบุรี': 'Prachinburi',
    'ระยอง': 'Rayong',
    'สระแก้ว': 'Sa Kaeo',
    'กาญจนบุรี': 'Kanchanaburi',
    'ตาก': 'Tak',
    'ประจวบคีรีขันธ์': 'Prachuap Khiri Khan',
    'เพชรบุรี': 'Phetchaburi',
    'ราชบุรี': 'Ratchaburi',
    'กระบี่': 'Krabi',
    'ชุมพร': 'Chumphon',
    'ตรัง': 'Trang',
    'นครศรีธรรมราช': 'Nakhon Si Thammarat',
    'นราธิวาส': 'Narathiwat',
    'ปัตตานี': 'Pattani',
    'พังงา': 'Phang Nga',
    'พัทลุง': 'Phatthalung',
    'ภูเก็ต': 'Phuket',
    'ระนอง': 'Ranong',
    'สตูล': 'Satun',
    'สงขลา': 'Songkhla',
    'สุราษฎร์ธานี': 'Surat Thani',
    'ยะลา': 'Yala'})

In [29]:
data_truck_60.head()

,Date,Time,Province,Loss,Party,Cause,Main cause of accidents
0,02/01/2560,11:16:00,Khon Kaen,3,รถอื่นๆ,รถบรรทุก,หลับใน
1,04/01/2560,07:30:00,Phrae,2,ไม่มีคู่กรณี,รถบรรทุก,หลับใน
2,04/01/2560,20:10:00,Saraburi,2,รถยนต์นั่งส่วนบุคคลเกิน7คน,รถยนต์นั่งส่วนบุคคลเกิน7คน,ขับแซงอย่างผิดกฎหมาย/แซงในที่คับขัน
3,04/01/2560,21:10:00,Phetchaburi,2,รถจักรยานยนต์,รถจักรยานยนต์,ขับแซงอย่างผิดกฎหมาย/แซงในที่คับขัน
4,05/01/2560,07:30:00,Chiang Rai,2,รถยนต์นั่งส่วนบุคคลเกิน7คน,รถยนต์นั่งส่วนบุคคลเกิน7คน,ขับรถเร็ว


In [30]:
#เปลี่ยนสาเหตุเป็นตัวเลขที่กำหนดไว้
data_truck_600 = data_truck_60
datamain60 = pd.DataFrame(data_truck_600)
datamain60['Main cause of accidents'] = datamain60['Main cause of accidents'].apply(lambda x: x.strip())
datamain60['Main cause of accidents'] = datamain60['Main cause of accidents'].replace({'ขับรถเร็ว':'M1',
'ขับรถตัดหน้าในระยะกระชั้นชิด':'M2',
'ขับตามหลังในระยะกระชั้นชิด':'M3',
'หลับใน':'M4',
' หลับใน':'M4',
'ขับแซงอย่างผิดกฎหมาย/แซงในที่คับขัน':	'M5',
'ไม่ชำนาญเส้นทาง':	'M6',
'ขาดทักษะที่ถูกต้องในการขับรถ':	'M7',
'ขับล้ำเข้าไปในช่องทางจราจรอื่น':	'M8',
'ขับล้ำเข้าไปในช่องทางฝั่งตรงข้าม':	'M9',
'ฝ่าฝืนเครื่องหมาย/สัญญาณจราจร':	'M10',
'เปลี่ยนช่องทางกระทันหัน':	'M11',
'เมาสุรา':	'M12',
'ใช้สารเสพติดออกฤทธิ์ต่อจิตประสาท':	'M13',
'ขาดสมาธิขณะขับรถ':	'M14',
'ขับย้อนศร':	'M15',
'จอดรถไหล่ทาง':	'M16',
'จอดในที่ห้ามจอด/จอดไหล่ทางล้ำ/จอดไม่ให้เครื่องหมายหรือสัญญาณ':	'M17',
'ทำกิจกรรมอื่นในขณะขับรถ (ก้มหยิบของ,โทรศัพท์)':	'M18',
'สภาพยางชำรุด':	'M19',
'ระบบเบรกชำรุด':	'M20',
'ระบบคันส่งคันชักชำรุด':	'M21',
'ระบบล้อและเพลาชำรุด':	'M22',
'ระบบไฟฟ้าชำรุด':	'M23',
'ระบบพวงมาลัยชำรุด':	'M24',
'อุปกรณ์ส่วนควบอื่นๆชำรุด':	'M25',
'สภาพถนนชำรุด':	'M26',
'ฝนตกถนนลื่น':	'M27',
'ถนนไม่มีไฟฟ้าแสงสว่าง':	'M28',
'ถนนมีสิ่งกีดขวางจราจร':	'M29',
'ประมาทร่วม':	'M30',
'รถอีกคันกระเด็นมาถูกโดยบังเอิญ':	'M31',
'ปัญหาสุขภาพผู้ขับรถ':	'M32',
'ไม่ระบุ':	'M33',
'คนเดินตัดหน้ารถ'	:'M34' })

In [31]:
data_truck_600.head()

,Date,Time,Province,Loss,Party,Cause,Main cause of accidents
0,02/01/2560,11:16:00,Khon Kaen,3,รถอื่นๆ,รถบรรทุก,M4
1,04/01/2560,07:30:00,Phrae,2,ไม่มีคู่กรณี,รถบรรทุก,M4
2,04/01/2560,20:10:00,Saraburi,2,รถยนต์นั่งส่วนบุคคลเกิน7คน,รถยนต์นั่งส่วนบุคคลเกิน7คน,M5
3,04/01/2560,21:10:00,Phetchaburi,2,รถจักรยานยนต์,รถจักรยานยนต์,M5
4,05/01/2560,07:30:00,Chiang Rai,2,รถยนต์นั่งส่วนบุคคลเกิน7คน,รถยนต์นั่งส่วนบุคคลเกิน7คน,M1


In [32]:
# เปลี่ยนตัวแปรคู่กรณี (Party) เป็นตัวเลขที่กำหนดไว้
data_truck_6000 = data_truck_600
datapar60 = pd.DataFrame(data_truck_6000)
datapar60['Party'] = datapar60['Party'].apply(lambda x: x.strip())
datapar60['Party'] = datapar60['Party'].replace({'รถยนต์บรรทุกส่วนบุคคล (ปิคอัพ)':	'P1',
'รถยนต์นั่งส่วนบุคคลไม่เกิน7คน':	'P2',
'รถยนต์นั่งส่วนบุคคลเกิน7คน':	'P3',
'รถบรรทุก':	'P4',
'รถโดยสาร':	'P5',
'รถจักรยานยนต์':	'P6',
'รถจักรยาน':	'P7',
'รถอื่นๆ':	'P8',
'คู่กรณีมากกว่า 1 คัน':	'P9',
'เสาไฟฟ้า':	'P10',
'ป้าย':'P11',
'ต้นไม้':'P12',
'ไม่มีคู่กรณี': 'P13',
'คนเดินเท้า':	'P14',
'ไม่ระบุ':	'P15' })

In [33]:
data_truck_6000.head()

,Date,Time,Province,Loss,Party,Cause,Main cause of accidents
0,02/01/2560,11:16:00,Khon Kaen,3,P8,รถบรรทุก,M4
1,04/01/2560,07:30:00,Phrae,2,P13,รถบรรทุก,M4
2,04/01/2560,20:10:00,Saraburi,2,P3,รถยนต์นั่งส่วนบุคคลเกิน7คน,M5
3,04/01/2560,21:10:00,Phetchaburi,2,P6,รถจักรยานยนต์,M5
4,05/01/2560,07:30:00,Chiang Rai,2,P3,รถยนต์นั่งส่วนบุคคลเกิน7คน,M1


In [34]:
# เปลี่ยนตัวแปรรถต้นเหตุ (Cause) เป็นตัวเลขที่กำหนดไว้
data_truck_60000 = data_truck_6000
datapar60 = pd.DataFrame(data_truck_60000)
datapar60['Cause'] = datapar60['Cause'].apply(lambda x: x.strip())
datapar60['Cause'] = datapar60['Cause'].replace({'รถยนต์บรรทุกส่วนบุคคล (ปิคอัพ)':	'C1',
'รถยนต์นั่งส่วนบุคคลไม่เกิน7คน':	'C2',
'รถยนต์นั่งส่วนบุคคลเกิน7คน':	'C3',
'รถบรรทุก':	'C4',
'รถโดยสาร':	'C5',
'รถจักรยานยนต์':	'C6',
'รถจักรยาน':	'C7',
'รถอื่นๆ':	'C8',
'คู่กรณีมากกว่า 1 คัน':	'C9',
'เสาไฟฟ้า':	'C10',
'ป้าย':'C11',
'ต้นไม้':'C12',
'ไม่มีคู่กรณี': 'C13',
'คนเดินเท้า':	'C14',
'ไม่ระบุ':	'C15'})

In [35]:
data_truck_60000.head()

,Date,Time,Province,Loss,Party,Cause,Main cause of accidents
0,02/01/2560,11:16:00,Khon Kaen,3,P8,C4,M4
1,04/01/2560,07:30:00,Phrae,2,P13,C4,M4
2,04/01/2560,20:10:00,Saraburi,2,P3,C3,M5
3,04/01/2560,21:10:00,Phetchaburi,2,P6,C6,M5
4,05/01/2560,07:30:00,Chiang Rai,2,P3,C3,M1


## Data truck 2561

In [36]:
data_2561 = pd.read_excel('/content/drive/MyDrive/bsc_dpdm_23/Final_Project/ข้อมูลอุบัติเหตุรถบรรทุก (5).xlsx', sheet_name = 2)
data_2561.head()

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,Date,Time,Province,Loss,Party,Cause,Main cause of accidents
0,01/01/2561,2.0,สมุทรสาคร,1,6,1,16
1,01/01/2561,2.0,ระยอง,1,6,1,16
2,02/01/2561,5.0,สมุทรสาคร,3,1,1,19
3,03/01/2561,6.0,สุราษฎร์ธานี,2,9,1,6
4,03/01/2561,1.0,ระยอง,1,6,6,1


In [37]:
#เปลี่ยนชื่อจังหวัดเป็นภาษาอังกฤษ
data_truck_61 = data_2561
datamain61 = pd.DataFrame(data_truck_61)
datamain61['Province'] = datamain61['Province'].apply(lambda x: x.strip())
datamain61['Province'] = datamain61['Province'].replace({
    'เชียงใหม่': 'Chiang Mai',
    'เชียงราย': 'Chiang Rai',
    'ลำปาง': 'Lampang',
    'ลำพูน': 'Lamphun',
    'แม่ฮ่องสอน': 'Mae Hong Son',
    'น่าน': 'Nan',
    'พะเยา': 'Phayao',
    'แพร่': 'Phrae',
    'อุตรดิตถ์': 'Uttaradit',
    'กาฬสินธุ์': 'Kalasin',
    'ขอนแก่น': 'Khon Kaen',
    'ชัยภูมิ': 'Chaiyaphum',
    'นครพนม': 'Nakhon Phanom',
    'นครราชสีมา': 'Nakhon Ratchasima',
    'บึงกาฬ': 'Bueng Kan',
    'บุรีรัมย์': 'Buriram',
    'มหาสารคาม': 'Maha Sarakham',
    'มุกดาหาร': 'Mukdahan',
    'ยโสธร': 'Yasothon',
    'ร้อยเอ็ด': 'Roi Et',
    'เลย': 'Loei',
    'สกลนคร': 'Sakon Nakhon',
    'สุรินทร์': 'Surin',
    'ศรีสะเกษ': 'Sisaket',
    'หนองคาย': 'Nong Khai',
    'หนองบัวลำภู': 'Nong Bua Lamphu',
    'อุดรธานี': 'Udon Thani',
    'อุบลราชธานี': 'Ubon Ratchathani',
    'อำนาจเจริญ': 'Amnat Charoen',
    'กรุงเทพมหานคร': 'Bangkok',
    'กำแพงเพชร': 'Kamphaeng Phet',
    'ชัยนาท': 'Chai Nat',
    'นครนายก': 'Nakhon Nayok',
    'นครปฐม': 'Nakhon Pathom',
    'นครสวรรค์': 'Nakhon Sawan',
    'นนทบุรี': 'Nonthaburi',
    'ปทุมธานี': 'Pathum Thani',
    'พระนครศรีอยุธยา': 'Phra Nakhon Si Ayutthaya',
    'พระนครศรี อยุธยา': 'Phra Nakhon Si Ayutthaya',
    'พิจิตร': 'Phichit',
    'พิษณุโลก': 'Phitsanulok',
    'เพชรบูรณ์': 'Phetchabun',
    'ลพบุรี': 'Lopburi',
    'สมุทรปราการ': 'Samut Prakan',
    'สมุทรสงคราม': 'Samut Songkhram',
    'สมุทรสาคร': 'Samut Sakhon',
    'สิงห์บุรี': 'Sing Buri',
    'สุโขทัย': 'Sukhothai',
    'สุพรรณบุรี': 'Suphan Buri',
    'สระบุรี': 'Saraburi',
    'อ่างทอง': 'Ang Thong',
    'อุทัยธานี': 'Uthai Thani',
    'จันทบุรี': 'Chanthaburi',
    'ฉะเชิงเทรา': 'Chachoengsao',
    'ชลบุรี': 'Chonburi',
    'ตราด': 'Trat',
    'ปราจีนบุรี': 'Prachinburi',
    'ระยอง': 'Rayong',
    'สระแก้ว': 'Sa Kaeo',
    'กาญจนบุรี': 'Kanchanaburi',
    'ตาก': 'Tak',
    'ประจวบคีรีขันธ์': 'Prachuap Khiri Khan',
    'เพชรบุรี': 'Phetchaburi',
    'ราชบุรี': 'Ratchaburi',
    'กระบี่': 'Krabi',
    'ชุมพร': 'Chumphon',
    'ตรัง': 'Trang',
    'นครศรีธรรมราช': 'Nakhon Si Thammarat',
    'นราธิวาส': 'Narathiwat',
    'ปัตตานี': 'Pattani',
    'พังงา': 'Phang Nga',
    'พัทลุง': 'Phatthalung',
    'ภูเก็ต': 'Phuket',
    'ระนอง': 'Ranong',
    'สตูล': 'Satun',
    'สงขลา': 'Songkhla',
    'สุราษฎร์ธานี': 'Surat Thani',
    'ยะลา': 'Yala'})


In [38]:
data_truck_61

,Date,Time,Province,Loss,Party,Cause,Main cause of accidents
0,01/01/2561,2.0,Samut Sakhon,1,6,1,16
1,01/01/2561,2.0,Rayong,1,6,1,16
2,02/01/2561,5.0,Samut Sakhon,3,1,1,19
3,03/01/2561,6.0,Surat Thani,2,9,1,6
4,03/01/2561,1.0,Rayong,1,6,6,1
...,...,...,...,...,...,...,...
853,29/12/2561,5.0,Phra Nakhon Si Ayutthaya,4,13,1,4
854,29/12/2561,5.0,Chanthaburi,2,9,1,3
855,29/12/2561,5.0,Lamphun,2,2,1,27
856,29/12/2561,6.0,Chachoengsao,3,1,1,4


In [39]:
data_truck_61['Party'] = data_truck_61['Party'].astype(str)

In [40]:
data_truck_611 = data_truck_61
datapar61 = pd.DataFrame(data_truck_611)
datapar61['Party'] = datapar61['Party'].apply(lambda x: x.strip())
datapar61['Party'] = datapar61['Party'].replace({'1':	'P1',
'2':	'P2',
'3':	'P3',
'4':	'P4',
'5':	'P5',
'6':	'P6',
'7':	'P7',
'8':	'P8',
'9':	'P9',
'10':	'P10',
'11':'P11',
'12':'P12',
'13': 'P13',
'14':	'P14',
'15':	'P15' })

In [41]:
data_truck_611.head()

,Date,Time,Province,Loss,Party,Cause,Main cause of accidents
0,01/01/2561,2.0,Samut Sakhon,1,P6,1,16
1,01/01/2561,2.0,Rayong,1,P6,1,16
2,02/01/2561,5.0,Samut Sakhon,3,P1,1,19
3,03/01/2561,6.0,Surat Thani,2,P9,1,6
4,03/01/2561,1.0,Rayong,1,P6,6,1


In [42]:
data_truck_611['Cause'] = data_truck_611['Cause'].astype(str)

In [43]:
data_truck_611 = data_truck_61
datacause61 = pd.DataFrame(data_truck_611)
datacause61['Cause'] = datacause61['Cause'].apply(lambda x: x.strip())
datacause61['Cause'] = datacause61['Cause'].replace({'1':	'C1',
'2':	'C2',
'3':	'C3',
'4':	'C4',
'5':	'C5',
'6':	'C6',
'7':	'C7',
'8':	'C8',
'9':	'C9',
'10':	'C10',
'11':'C11',
'12':'C12',
'13': 'C13',
'14':	'C14',
'15':	'C15'})


In [44]:
data_truck_611

,Date,Time,Province,Loss,Party,Cause,Main cause of accidents
0,01/01/2561,2.0,Samut Sakhon,1,P6,C1,16
1,01/01/2561,2.0,Rayong,1,P6,C1,16
2,02/01/2561,5.0,Samut Sakhon,3,P1,C1,19
3,03/01/2561,6.0,Surat Thani,2,P9,C1,6
4,03/01/2561,1.0,Rayong,1,P6,C6,1
...,...,...,...,...,...,...,...
853,29/12/2561,5.0,Phra Nakhon Si Ayutthaya,4,P13,C1,4
854,29/12/2561,5.0,Chanthaburi,2,P9,C1,3
855,29/12/2561,5.0,Lamphun,2,P2,C1,27
856,29/12/2561,6.0,Chachoengsao,3,P1,C1,4


In [45]:
data_truck_611['Main cause of accidents'] = data_truck_611['Main cause of accidents'].astype(str)

In [46]:
data_truck_611 = data_truck_61
datamain61 = pd.DataFrame(data_truck_611)
datamain61['Main cause of accidents'] = datamain61['Main cause of accidents'].apply(lambda x: x.strip())
datamain61['Main cause of accidents'] = datamain61['Main cause of accidents'].replace({'1':'M1',
'2':'M2',
'3':'M3',
'4':'M4',
'5':	'M5',
'6':	'M6',
'7':	'M7',
'8':	'M8',
'9':	'M9',
'10':	'M10',
'11':	'M11',
'12':	'M12',
'13':	'M13',
'14':	'M14',
'15':	'M15',
'16':	'M16',
'17':	'M17',
'18':	'M18',
'19':	'M19',
'20':	'M20',
'21':	'M21',
'22':	'M22',
'23':	'M23',
'24':	'M24',
'25':	'M25',
'26':	'M26',
'27':	'M27',
'28':	'M28',
'29':	'M29',
'30':	'M30',
'31':	'M31',
'32':	'M32',
'33':	'M33',
'34'	:'M34' })


In [47]:
data_truck_611.head()

,Date,Time,Province,Loss,Party,Cause,Main cause of accidents
0,01/01/2561,2.0,Samut Sakhon,1,P6,C1,M16
1,01/01/2561,2.0,Rayong,1,P6,C1,M16
2,02/01/2561,5.0,Samut Sakhon,3,P1,C1,M19
3,03/01/2561,6.0,Surat Thani,2,P9,C1,M6
4,03/01/2561,1.0,Rayong,1,P6,C6,M1


## Data truck 2562

In [48]:
data_2562 = pd.read_excel('/content/drive/MyDrive/bsc_dpdm_23/Final_Project/ข้อมูลอุบัติเหตุรถบรรทุก (5).xlsx', sheet_name = 3)
data_2562.head()

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,Date,Time,Province,Loss,Party,Cause,Main cause of accidents
0,01/01/2562,12:45:00,สิงห์บุรี,1,รถจักรยานยนต์,รถบรรทุก,จอดรถไหล่ทาง
1,01/01/2562,05:00:00,สุพรรณบุรี,2,รถโดยสาร,รถบรรทุก,ขับย้อนศร
2,01/01/2562,12:50:00,พระนครศรีอยุธยา,4,คู่กรณีมากกว่า 1 คัน,รถบรรทุก,ขับตามหลังในระยะกระชั้นชิด
3,02/01/2562,13:30:00,เลย,3,รถยนต์นั่งส่วนบุคคลไม่เกิน7คน,รถบรรทุก,ขับรถตัดหน้าในระยะกระชั้นชิด
4,02/01/2562,17:00:00,ระนอง,1,รถจักรยานยนต์,รถจักรยานยนต์,ขับล้ำเข้าไปในช่องทางจราจรอื่น


In [49]:
#เปลี่ยนชื่อจังหวัดเป็นภาษาอังกฤษ
data_truck_62 = data_2562
data_2562 = pd.DataFrame(data_truck_62)
data_2562['Province'] = data_2562['Province'].apply(lambda x: x.strip())
data_2562['Province'] = data_2562['Province'].replace({
    'เชียงใหม่': 'Chiang Mai',
    'เชียงราย': 'Chiang Rai',
    'ลำปาง': 'Lampang',
    'ลำพูน': 'Lamphun',
    'แม่ฮ่องสอน': 'Mae Hong Son',
    'น่าน': 'Nan',
    'พะเยา': 'Phayao',
    'แพร่': 'Phrae',
    'อุตรดิตถ์': 'Uttaradit',
    'กาฬสินธุ์': 'Kalasin',
    'ขอนแก่น': 'Khon Kaen',
    'ชัยภูมิ': 'Chaiyaphum',
    'นครพนม': 'Nakhon Phanom',
    'นครราชสีมา': 'Nakhon Ratchasima',
    'บึงกาฬ': 'Bueng Kan',
    'บุรีรัมย์': 'Buriram',
    'มหาสารคาม': 'Maha Sarakham',
    'มุกดาหาร': 'Mukdahan',
    'ยโสธร': 'Yasothon',
    'ร้อยเอ็ด': 'Roi Et',
    'เลย': 'Loei',
    'สกลนคร': 'Sakon Nakhon',
    'สุรินทร์': 'Surin',
    'ศรีสะเกษ': 'Sisaket',
    'หนองคาย': 'Nong Khai',
    'หนองบัวลำภู': 'Nong Bua Lamphu',
    'อุดรธานี': 'Udon Thani',
    'อุบลราชธานี': 'Ubon Ratchathani',
    'อำนาจเจริญ': 'Amnat Charoen',
    'กทม.': 'Bangkok',
    'กรุงเทพฯ': 'Bangkok',
    'กรุงเทพมหานคร': 'Bangkok',
    'กำแพงเพชร': 'Kamphaeng Phet',
    'ชัยนาท': 'Chai Nat',
    'นครนายก': 'Nakhon Nayok',
    'นครปฐม': 'Nakhon Pathom',
    'นครสวรรค์': 'Nakhon Sawan',
    'นนทบุรี': 'Nonthaburi',
    'ปทุมธานี': 'Pathum Thani',
    'พระนครศรีอยุธยา': 'Phra Nakhon Si Ayutthaya',
    'พระนครศรี อยุธยา': 'Phra Nakhon Si Ayutthaya',
    'พิจิตร': 'Phichit',
    'พิษณุโลก': 'Phitsanulok',
    'เพชรบูรณ์': 'Phetchabun',
    'ลพบุรี': 'Lopburi',
    'สมุทรปราการ': 'Samut Prakan',
    'สมุทรสงคราม': 'Samut Songkhram',
    'สมุทรสาคร': 'Samut Sakhon',
    'สิงห์บุรี': 'Sing Buri',
    'สุโขทัย': 'Sukhothai',
    'สุพรรณบุรี': 'Suphan Buri',
    'สระบุรี': 'Saraburi',
    'อ่างทอง': 'Ang Thong',
    'อุทัยธานี': 'Uthai Thani',
    'จันทบุรี': 'Chanthaburi',
    'ฉะเชิงเทรา': 'Chachoengsao',
    'ชลบุรี': 'Chonburi',
    'ตราด': 'Trat',
    'ปราจีนบุรี': 'Prachinburi',
    'ระยอง': 'Rayong',
    'สระแก้ว': 'Sa Kaeo',
    'กาญจนบุรี': 'Kanchanaburi',
    'ตาก': 'Tak',
    'ประจวบคีรีขันธ์': 'Prachuap Khiri Khan',
    'เพชรบุรี': 'Phetchaburi',
    'ราชบุรี': 'Ratchaburi',
    'กระบี่': 'Krabi',
    'ชุมพร': 'Chumphon',
    'ตรัง': 'Trang',
    'นครศรีธรรมราช': 'Nakhon Si Thammarat',
    'นราธิวาส': 'Narathiwat',
    'ปัตตานี': 'Pattani',
    'พังงา': 'Phang Nga',
    'พัทลุง': 'Phatthalung',
    'ภูเก็ต': 'Phuket',
    'ระนอง': 'Ranong',
    'สตูล': 'Satun',
    'สงขลา': 'Songkhla',
    'สุราษฎร์ธานี': 'Surat Thani',
    'ยะลา': 'Yala'})


In [50]:
data_truck_62.head()

,Date,Time,Province,Loss,Party,Cause,Main cause of accidents
0,01/01/2562,12:45:00,Sing Buri,1,รถจักรยานยนต์,รถบรรทุก,จอดรถไหล่ทาง
1,01/01/2562,05:00:00,Suphan Buri,2,รถโดยสาร,รถบรรทุก,ขับย้อนศร
2,01/01/2562,12:50:00,Phra Nakhon Si Ayutthaya,4,คู่กรณีมากกว่า 1 คัน,รถบรรทุก,ขับตามหลังในระยะกระชั้นชิด
3,02/01/2562,13:30:00,Loei,3,รถยนต์นั่งส่วนบุคคลไม่เกิน7คน,รถบรรทุก,ขับรถตัดหน้าในระยะกระชั้นชิด
4,02/01/2562,17:00:00,Ranong,1,รถจักรยานยนต์,รถจักรยานยนต์,ขับล้ำเข้าไปในช่องทางจราจรอื่น


In [51]:
#เปลี่ยนสาเหตุเป็นตัวเลขที่กำหนดไว้
data_truck_622 = data_truck_62
datamain_62 = pd.DataFrame(data_truck_622)
datamain_62['Main cause of accidents'] = datamain_62['Main cause of accidents'].apply(lambda x: x.strip())
datamain_62['Main cause of accidents'] = datamain_62['Main cause of accidents'].replace({'ขับรถเร็ว':'M1',
'ขับรถตัดหน้าในระยะกระชั้นชิด':'M2',
'ขับตามหลังในระยะกระชั้นชิด':'M3',
'หลับใน':'M4',
' หลับใน':'M4',
'ขับแซงอย่างผิดกฎหมาย/แซงในที่คับขัน':	'M5',
'ไม่ชำนาญเส้นทาง':	'M6',
'ขาดทักษะที่ถูกต้องในการขับรถ':	'M7',
'ขับล้ำเข้าไปในช่องทางจราจรอื่น':	'M8',
'ขับล้ำเข้าไปในช่องทางฝั่งตรงข้าม':	'M9',
'ฝ่าฝืนเครื่องหมาย/สัญญาณจราจร':	'M10',
'เปลี่ยนช่องทางกระทันหัน':	'M11',
'เมาสุรา':	'M12',
'ใช้สารเสพติดออกฤทธิ์ต่อจิตประสาท':	'M13',
'ขาดสมาธิขณะขับรถ':	'M14',
'ขับย้อนศร':	'M15',
'จอดรถไหล่ทาง':	'M16',
'จอดในที่ห้ามจอด/จอดไหล่ทางล้ำ/จอดไม่ให้เครื่องหมายหรือสัญญาณ':	'M17',
'ทำกิจกรรมอื่นในขณะขับรถ (ก้มหยิบของ,โทรศัพท์)':	'M18',
'สภาพยางชำรุด':	'M19',
'ระบบเบรกชำรุด':	'M20',
'ระบบคันส่งคันชักชำรุด':	'M21',
'ระบบล้อและเพลาชำรุด':	'M22',
'ระบบไฟฟ้าชำรุด':	'M23',
'ระบบพวงมาลัยชำรุด':	'M24',
'อุปกรณ์ส่วนควบอื่นๆชำรุด':	'M25',
'สภาพถนนชำรุด':	'M26',
'ฝนตกถนนลื่น':	'M27',
'ถนนไม่มีไฟฟ้าแสงสว่าง':	'M28',
'ถนนมีสิ่งกีดขวางจราจร':	'M29',
'ประมาทร่วม':	'M30',
'รถอีกคันกระเด็นมาถูกโดยบังเอิญ':	'M31',
'ปัญหาสุขภาพผู้ขับรถ':	'M32',
'ไม่ระบุ':	'M33',
'คนเดินตัดหน้ารถ'	:'M34' })

In [52]:
data_truck_622.head()

,Date,Time,Province,Loss,Party,Cause,Main cause of accidents
0,01/01/2562,12:45:00,Sing Buri,1,รถจักรยานยนต์,รถบรรทุก,M16
1,01/01/2562,05:00:00,Suphan Buri,2,รถโดยสาร,รถบรรทุก,M15
2,01/01/2562,12:50:00,Phra Nakhon Si Ayutthaya,4,คู่กรณีมากกว่า 1 คัน,รถบรรทุก,M3
3,02/01/2562,13:30:00,Loei,3,รถยนต์นั่งส่วนบุคคลไม่เกิน7คน,รถบรรทุก,M2
4,02/01/2562,17:00:00,Ranong,1,รถจักรยานยนต์,รถจักรยานยนต์,M8


In [53]:
# เปลี่ยนตัวแปรคู่กรณี (Party) เป็นตัวเลขที่กำหนดไว้
data_truck_6222 = data_truck_622
datapar62 = pd.DataFrame(data_truck_6222)
datapar62['Party'] = datapar62['Party'].apply(lambda x: x.strip())
datapar62['Party'] = datapar62['Party'].replace({'รถยนต์บรรทุกส่วนบุคคล (ปิคอัพ)':	'P1',
'รถยนต์นั่งส่วนบุคคลไม่เกิน7คน':	'P2',
'รถยนต์นั่งส่วนบุคคลเกิน7คน':	'P3',
'รถบรรทุก':	'P4',
'รถโดยสาร':	'P5',
'รถจักรยานยนต์':	'P6',
'รถจักรยาน':	'P7',
'รถอื่นๆ':	'P8',
'คู่กรณีมากกว่า 1 คัน':	'P9',
'เสาไฟฟ้า':	'P10',
'ป้าย':'P11',
'ต้นไม้':'P12',
'ไม่มีคู่กรณี': 'P13',
'คนเดินเท้า':	'P14',
'ไม่ระบุ':	'P15' })

In [54]:
data_truck_6222.head()

,Date,Time,Province,Loss,Party,Cause,Main cause of accidents
0,01/01/2562,12:45:00,Sing Buri,1,P6,รถบรรทุก,M16
1,01/01/2562,05:00:00,Suphan Buri,2,P5,รถบรรทุก,M15
2,01/01/2562,12:50:00,Phra Nakhon Si Ayutthaya,4,P9,รถบรรทุก,M3
3,02/01/2562,13:30:00,Loei,3,P2,รถบรรทุก,M2
4,02/01/2562,17:00:00,Ranong,1,P6,รถจักรยานยนต์,M8


In [55]:
data_truck_62222 = data_truck_6222
datacause_62 = pd.DataFrame(data_truck_62222)
datacause_62['Cause'] = datacause_62['Cause'].apply(lambda x: x.strip())
datacause_62['Cause'] = datacause_62['Cause'].replace({'รถยนต์บรรทุกส่วนบุคคล (ปิคอัพ)':	'C1',
'รถยนต์นั่งส่วนบุคคลไม่เกิน7คน':	'C2',
'รถยนต์นั่งส่วนบุคคลเกิน7คน':	'C3',
'รถบรรทุก':	'C4',
'รถโดยสาร':	'C5',
'รถจักรยานยนต์':	'C6',
'รถจักรยาน':	'C7',
'รถอื่นๆ':	'C8',
'คู่กรณีมากกว่า 1 คัน':	'C9',
'เสาไฟฟ้า':	'C10',
'ป้าย':'C11',
'ต้นไม้':'C12',
'ไม่มีคู่กรณี': 'C13',
'คนเดินเท้า':	'C14',
'ไม่ระบุ':	'C15'})

In [56]:
data_truck_62222.head()

,Date,Time,Province,Loss,Party,Cause,Main cause of accidents
0,01/01/2562,12:45:00,Sing Buri,1,P6,C4,M16
1,01/01/2562,05:00:00,Suphan Buri,2,P5,C4,M15
2,01/01/2562,12:50:00,Phra Nakhon Si Ayutthaya,4,P9,C4,M3
3,02/01/2562,13:30:00,Loei,3,P2,C4,M2
4,02/01/2562,17:00:00,Ranong,1,P6,C6,M8


#Change Time

## เปลี่ยนช่วงเวลาdata truck 2560

---



In [57]:
k=[]
L=[]
for i in range(len(data_truck_60000)):
  if type(data_truck_60000['Time'][i])==str:
    k.append(i)
  else:
    L.append(i)

In [58]:
data_truck_60000['Time'][476]

datetime.time(21, 21)

In [59]:
data_truck_60000=data_truck_60000.replace(data_truck_60000.iloc[476,1],np.nan)

In [60]:
data_truck_60000.iloc[476,1]

nan

In [61]:
import datetime
data_truck_60000.replace(data_truck_60000.iloc[476,1],np.nan)
def time_in_range(start, end, x):
    if start <= end:
        return start <= x <= end
    else:
        return start <= x or x <= end

#data_truck_60['Time'] = pd.to_datetime(data_truck_60['Time'], format='%I:%M:%S' or '%I:%M:%S %p', errors='coerce').dt.time
time_labels = []
how = []

start_1 = datetime.time(16, 1, 0)
end_1 = datetime.time(20, 0, 59)

start_2 = datetime.time(20, 1, 0)
end_2 = datetime.time(0, 0, 59)

start_3 = datetime.time(0, 1, 0)
end_3 = datetime.time(4, 0, 59)

start_4 = datetime.time(4, 1, 0)
end_4 = datetime.time(8, 0, 59)

start_5 = datetime.time(8, 1, 0)
end_5 = datetime.time(12, 0, 59)

start_6 = datetime.time(12, 1, 0)
end_6 = datetime.time(16, 0, 0)

filtered_data = data_truck_60.dropna(subset=['Time'])

for i in range(len(filtered_data['Time'])):
    if time_in_range(start_1, end_1, filtered_data['Time'].iloc[i])== True:
        time_labels.append(1)
    elif time_in_range(start_2, end_2, filtered_data['Time'].iloc[i])== True:
        time_labels.append(2)
    elif time_in_range(start_3, end_3, filtered_data['Time'].iloc[i])== True:
        time_labels.append(3)
    elif time_in_range(start_4, end_4, filtered_data['Time'].iloc[i])== True:
        time_labels.append(4)
    elif time_in_range(start_5, end_5, filtered_data['Time'].iloc[i])== True:
        time_labels.append(5)
    elif time_in_range(start_6, end_6, filtered_data['Time'].iloc[i])== True:
        time_labels.append(6)
    else:
        how.append(i)

print(time_labels)

[5, 4, 2, 2, 4, 3, 5, 4, 6, 1, 2, 4, 5, 1, 3, 4, 6, 6, 3, 2, 5, 1, 4, 3, 4, 2, 4, 4, 4, 4, 1, 1, 2, 2, 5, 1, 3, 2, 6, 2, 5, 4, 2, 6, 3, 5, 1, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 3, 4, 5, 4, 4, 6, 4, 4, 4, 5, 5, 5, 4, 5, 5, 5, 4, 5, 5, 5, 3, 5, 4, 6, 4, 5, 4, 4, 5, 6, 1, 2, 2, 2, 2, 6, 5, 1, 5, 1, 2, 3, 2, 4, 4, 6, 1, 2, 1, 1, 2, 3, 5, 1, 2, 2, 4, 5, 1, 2, 4, 1, 1, 5, 4, 4, 5, 6, 5, 1, 2, 3, 5, 5, 1, 2, 4, 5, 5, 5, 1, 2, 5, 6, 6, 6, 1, 2, 5, 5, 6, 5, 6, 6, 5, 5, 1, 1, 6, 2, 5, 6, 6, 1, 2, 3, 4, 1, 6, 4, 5, 6, 6, 2, 6, 5, 2, 6, 4, 1, 5, 2, 2, 5, 2, 2, 3, 4, 6, 2, 5, 1, 2, 4, 3, 5, 4, 3, 4, 4, 5, 5, 3, 5, 6, 5, 6, 5, 3, 5, 4, 4, 5, 5, 5, 4, 5, 4, 4, 4, 5, 5, 5, 4, 5, 4, 3, 6, 3, 5, 4, 5, 3, 4, 5, 5, 3, 4, 5, 4, 5, 5, 4, 3, 5, 5, 4, 5, 5, 3, 5, 5, 5, 4, 5, 4, 4, 4, 4, 4, 4, 3, 5, 4, 4, 5, 4, 5, 4, 4, 4, 4, 5, 5, 4, 5, 6, 5, 6, 5, 3, 4, 4, 5, 4, 5, 3, 5, 4, 4, 4, 4, 1, 3, 4, 5, 2, 3, 6, 6, 4, 5, 3, 4, 2, 2, 4, 4, 5, 1, 1, 2, 4, 4, 6, 6, 6, 2, 6, 5, 5, 5, 6, 1, 2, 4, 5, 6, 3, 3, 3, 4, 

In [62]:
time_labels = pd.DataFrame({
    'Time': [5, 4, 2, 2, 4, 3, 5, 4, 6, 1, 2, 4, 5, 1, 3, 4, 6, 6, 3, 2, 5, 1, 4, 3, 4, 2, 4, 4, 4, 4, 1, 1, 2, 2, 5, 1, 3, 2, 6, 2, 5, 4, 2, 6, 3, 5, 1, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 3, 4, 5, 4, 4, 6, 4, 4, 4, 5, 5, 5, 4, 5, 5, 5, 4, 5, 5, 5, 3, 5, 4, 6, 4, 5, 4, 4, 5, 6, 1, 2, 2, 2, 2, 6, 5, 1, 5, 1, 2, 3, 2, 4, 4, 6, 1, 2, 1, 1, 2, 3, 5, 1, 2, 2, 4, 5, 1, 2, 4, 1, 1, 5, 4, 4, 5, 6, 5, 1, 2, 3, 5, 5, 1, 2, 4, 5, 5, 5, 1, 2, 5, 6, 6, 6, 1, 2, 5, 5, 6, 5, 6, 6, 5, 5, 1, 1, 6, 2, 5, 6, 6, 1, 2, 3, 4, 1, 6, 4, 5, 6, 6, 2, 6, 5, 2, 6, 4, 1, 5, 2, 2, 5, 2, 2, 3, 4, 6, 2, 5, 1, 2, 4, 3, 5, 4, 3, 4, 4, 5, 5, 3, 5, 6, 5, 6, 5, 3, 5, 4, 4, 5, 5, 5, 4, 5, 4, 4, 4, 5, 5, 5, 4, 5, 4, 3, 6, 3, 5, 4, 5, 3, 4, 5, 5, 3, 4, 5, 4, 5, 5, 4, 3, 5, 5, 4, 5, 5, 3, 5, 5, 5, 4, 5, 4, 4, 4, 4, 4, 4, 3, 5, 4, 4, 5, 4, 5, 4, 4, 4, 4, 5, 5, 4, 5, 6, 5, 6, 5, 3, 4, 4, 5, 4, 5, 3, 5, 4, 4, 4, 4, 1, 3, 4, 5, 2, 3, 6, 6, 4, 5, 3, 4, 2, 2, 4, 4, 5, 1, 1, 2, 4, 4, 6, 6, 6, 2, 6, 5, 5, 5, 6, 1, 2, 4, 5, 6, 3, 3, 3, 4, 5, 6, 6, 6, 4, 6, 1, 2, 1, 3, 4, 6, 6, 5, 1, 1, 4, 5, 2, 2, 4, 4, 6, 1, 3, 5, 6, 3, 6, 6, 3, 5, 6, 3, 4, 4, 3, 6, 1, 2, 2, 1, 2, 6, 6, 1, 2, 4, 4, 5, 1, 2, 3, 5, 5, 6, 6, 3, 4, 5, 2, 4, 5, 5, 6, 6, 1, 1, 6, 6, 3, 5, 1, 4, 4, 2, 4, 5, 6, 6, 1, 2, 4, 2, 4, 1, 2, 2, 4, 5, 6, 5, 4, 6, 5, 6, 6, 2, 6, 1, 4, 4, 4, 5, 5, 1, 4, 6, 1, 6, 4, 1, 3, 4, 4, 5, 5, 6, 3, 3, 2, 4, 3, 4, 5, 3, 6, 2, 4, 4, 4, 5, 6, 1, 2, 6, 1, 1, 6, 3, 5, 5, 6, 2, 2, 4, 5, 2, 5, 1, 1, 3, 5, 2, 2, 6, 4, 1, 1, 4, 1, 3, 3, 5, 6, 2, 4, 6, 2, 2, 3, 4, 5, 3, 3, 4, 5, 2, 6, 2, 4, 1, 1, 2, 5, 1, 3, 4, 4, 6, 6, 6, 5, 5, 6, 5, 5, 2, 1, 2, 3, 6, 3, 3, 2, 4, 5, 4, 5, 6, 4, 4, 5, 5, 6, 6, 6, 1, 1, 6, 6, 2, 2, 3, 3, 3, 6, 6, 2, 2, 2, 2, 4, 5, 1, 2, 5, 3, 4, 5, 1, 1, 6, 4, 3, 5, 6, 6, 2, 3, 1, 2, 4, 2, 4, 1, 1, 3, 4, 1, 1, 2, 3, 3, 3, 6, 6, 4, 4, 6, 6, 6, 1, 1, 1, 3, 3, 5, 4, 5, 6, 6, 2, 3, 5, 1, 4, 6, 5, 5, 4, 4, 5, 5, 6, 6, 3, 4, 6, 1, 3, 4, 5, 1, 6, 6, 2, 6, 6, 4, 4, 5, 5, 1, 1, 1, 4, 1, 3, 4, 5, 6, 2, 3, 4, 6, 1, 6, 3, 4, 4, 6, 3, 6, 6, 4, 6, 4, 6, 6, 5, 4, 6, 4, 1, 5, 2, 6, 6, 4, 6, 4, 5, 6, 6, 3, 5, 6, 6, 1, 2, 3, 4, 6, 1, 3, 3, 6, 6, 6, 4, 4, 4, 1, 4, 6, 5, 2, 2, 6, 6, 4, 4, 1, 6, 3, 6, 4, 4, 5, 4, 1, 5, 5, 6, 1, 3, 4, 6, 6, 6, 6, 4, 2, 2, 1, 3, 4, 5, 1, 6, 4, 4, 1, 6, 4, 1, 4, 5, 6, 1, 2, 4, 6, 4, 6, 6, 6, 1, 6, 1, 5, 6, 5, 2, 2, 4, 4, 5, 1, 2, 3, 3, 6, 6, 2, 6, 6, 3, 1, 1, 1, 3, 2, 2, 4, 1, 4, 2, 4, 4, 6, 3, 5, 5, 2, 3, 4, 4, 5, 6, 1, 1, 5, 5, 6, 4, 5, 6, 6, 6, 2, 3, 4, 4, 4, 6, 2, 6, 1, 5, 5, 5, 5, 1, 4, 4, 4, 5, 5, 6, 2, 6, 2, 1, 1, 3, 3, 5, 5, 5, 1, 2, 2, 4, 5, 6, 1, 5, 1, 1, 4, 2, 6, 5, 4, 4, 5, 6, 6, 1, 3, 4, 6, 6, 2, 5, 5, 6, 6, 1, 2, 6, 1, 2, 2, 1, 2, 6, 1],})

In [63]:
data_truck_60000['Time'] = time_labels
print(data_truck_60000)

           Date  Time             Province  Loss Party Cause  \
0    02/01/2560     5            Khon Kaen     3    P8    C4   
1    04/01/2560     4                Phrae     2   P13    C4   
2    04/01/2560     2             Saraburi     2    P3    C3   
3    04/01/2560     2          Phetchaburi     2    P6    C6   
4    05/01/2560     4           Chiang Rai     2    P3    C3   
..          ...   ...                  ...   ...   ...   ...   
877  29/12/2560     2         Samut Prakan     3    P6    C6   
878  30/12/2560     2             Chonburi     2    P6    C6   
879  30/12/2560     1              Lampang     1    P6    C6   
880  31/12/2560     2  Prachuap Khiri Khan     1    P6    C6   
881  31/12/2560     6                 Trat     1   P14   C14   

    Main cause of accidents  
0                        M4  
1                        M4  
2                        M5  
3                        M5  
4                        M1  
..                      ...  
877                  

In [64]:
data_truck_60000

,Date,Time,Province,Loss,Party,Cause,Main cause of accidents
0,02/01/2560,5,Khon Kaen,3,P8,C4,M4
1,04/01/2560,4,Phrae,2,P13,C4,M4
2,04/01/2560,2,Saraburi,2,P3,C3,M5
3,04/01/2560,2,Phetchaburi,2,P6,C6,M5
4,05/01/2560,4,Chiang Rai,2,P3,C3,M1
...,...,...,...,...,...,...,...
877,29/12/2560,2,Samut Prakan,3,P6,C6,M3
878,30/12/2560,2,Chonburi,2,P6,C6,M2
879,30/12/2560,1,Lampang,1,P6,C6,M2
880,31/12/2560,2,Prachuap Khiri Khan,1,P6,C6,M2


In [65]:
data_truck_60000.replace({np.nan: '-'}, inplace=True)
print(data_truck_60000)

           Date  Time             Province  Loss Party Cause  \
0    02/01/2560     5            Khon Kaen     3    P8    C4   
1    04/01/2560     4                Phrae     2   P13    C4   
2    04/01/2560     2             Saraburi     2    P3    C3   
3    04/01/2560     2          Phetchaburi     2    P6    C6   
4    05/01/2560     4           Chiang Rai     2    P3    C3   
..          ...   ...                  ...   ...   ...   ...   
877  29/12/2560     2         Samut Prakan     3    P6    C6   
878  30/12/2560     2             Chonburi     2    P6    C6   
879  30/12/2560     1              Lampang     1    P6    C6   
880  31/12/2560     2  Prachuap Khiri Khan     1    P6    C6   
881  31/12/2560     6                 Trat     1   P14   C14   

    Main cause of accidents  
0                        M4  
1                        M4  
2                        M5  
3                        M5  
4                        M1  
..                      ...  
877                  

In [66]:
# ลบจุดทศนิยม
data_truck_60000['Time'] = data_truck_60000['Time'].astype(str)
data_truck_60000['Time'] = data_truck_60000['Time'].apply(lambda x: x.strip("0. "))

In [67]:
data_truck_60000

,Date,Time,Province,Loss,Party,Cause,Main cause of accidents
0,02/01/2560,5,Khon Kaen,3,P8,C4,M4
1,04/01/2560,4,Phrae,2,P13,C4,M4
2,04/01/2560,2,Saraburi,2,P3,C3,M5
3,04/01/2560,2,Phetchaburi,2,P6,C6,M5
4,05/01/2560,4,Chiang Rai,2,P3,C3,M1
...,...,...,...,...,...,...,...
877,29/12/2560,2,Samut Prakan,3,P6,C6,M3
878,30/12/2560,2,Chonburi,2,P6,C6,M2
879,30/12/2560,1,Lampang,1,P6,C6,M2
880,31/12/2560,2,Prachuap Khiri Khan,1,P6,C6,M2


In [68]:
# เปลี่ยนเวลาจากตัวเลขให้เป็นช่วง

data_truck_600001 = data_truck_60000
data_truck_60000 = pd.DataFrame(data_truck_600001)
data_truck_60000['Time'] = data_truck_60000['Time'].apply(lambda x: x.strip())
data_truck_60000['Time'] = data_truck_60000['Time'].replace({'1': '16.01 - 20.00',
'2':'20.01 - 00.00',
'3':'00.01 - 04.00',
'4': '04.01 - 08.00',
'5': '08.01 - 12.00',
'6':'12.01 - 16.00'})

In [69]:
data_truck_600001

,Date,Time,Province,Loss,Party,Cause,Main cause of accidents
0,02/01/2560,08.01 - 12.00,Khon Kaen,3,P8,C4,M4
1,04/01/2560,04.01 - 08.00,Phrae,2,P13,C4,M4
2,04/01/2560,20.01 - 00.00,Saraburi,2,P3,C3,M5
3,04/01/2560,20.01 - 00.00,Phetchaburi,2,P6,C6,M5
4,05/01/2560,04.01 - 08.00,Chiang Rai,2,P3,C3,M1
...,...,...,...,...,...,...,...
877,29/12/2560,20.01 - 00.00,Samut Prakan,3,P6,C6,M3
878,30/12/2560,20.01 - 00.00,Chonburi,2,P6,C6,M2
879,30/12/2560,16.01 - 20.00,Lampang,1,P6,C6,M2
880,31/12/2560,20.01 - 00.00,Prachuap Khiri Khan,1,P6,C6,M2


## เปลี่ยนช่วงเวลา 2561

In [70]:
data_truck_611.head()

,Date,Time,Province,Loss,Party,Cause,Main cause of accidents
0,01/01/2561,2.0,Samut Sakhon,1,P6,C1,M16
1,01/01/2561,2.0,Rayong,1,P6,C1,M16
2,02/01/2561,5.0,Samut Sakhon,3,P1,C1,M19
3,03/01/2561,6.0,Surat Thani,2,P9,C1,M6
4,03/01/2561,1.0,Rayong,1,P6,C6,M1


In [71]:
# ลบจุดทศนิยม
data_truck_611['Time'] = data_truck_611['Time'].astype(str)
data_truck_611['Time'] = data_truck_611['Time'].apply(lambda x: x.strip("0. "))

In [72]:
data_truck_6111 = data_truck_611
data_truck_611 = pd.DataFrame(data_truck_6111)
data_truck_611['Time'] = data_truck_611['Time'].apply(lambda x: x.strip())
data_truck_611['Time'] = data_truck_611['Time'].replace({'1': '16.01 - 20.00',
'2':'20.01 - 00.00',
'3':'00.01 - 04.00',
'4': '04.01 - 08.00',
'5': '08.01 - 12.00',
'6':'12.01 - 16.00'})

In [73]:
data_truck_6111.head()

,Date,Time,Province,Loss,Party,Cause,Main cause of accidents
0,01/01/2561,20.01 - 00.00,Samut Sakhon,1,P6,C1,M16
1,01/01/2561,20.01 - 00.00,Rayong,1,P6,C1,M16
2,02/01/2561,08.01 - 12.00,Samut Sakhon,3,P1,C1,M19
3,03/01/2561,12.01 - 16.00,Surat Thani,2,P9,C1,M6
4,03/01/2561,16.01 - 20.00,Rayong,1,P6,C6,M1


## เปลี่ยนช่วงเวลา data truck 2562

In [74]:
k=[]
L=[]
for i in range(len(data_truck_62222)):
  if type(data_truck_62222['Time'][i])==str:
    k.append(i)
  else:
    L.append(i)

In [75]:
import datetime

def time_in_range(start, end, x):
    if start <= end:
        return start <= x <= end
    else:
        return start <= x or x <= end
time_labels = []
how = []
data_truck_62222['Time'] = pd.to_datetime(data_truck_62222['Time'], format='%I:%M:%S' or '%I:%M:%S %p', errors='coerce').dt.time


start_1 = datetime.time(16, 1, 0)
end_1 = datetime.time(20, 0, 59)

start_2 = datetime.time(20, 1, 0)
end_2 = datetime.time(0, 0, 59)

start_3 = datetime.time(0, 1, 0)
end_3 = datetime.time(4, 0, 59)

start_4 = datetime.time(4, 1, 0)
end_4 = datetime.time(8, 0, 59)

start_5 = datetime.time(8, 1, 0)
end_5 = datetime.time(12, 0, 59)

start_6 = datetime.time(12, 1, 0)
end_6 = datetime.time(16, 0, 0)
filtered_data = data_truck_62.dropna(subset=['Time'])

for i in range(len(filtered_data['Time'])):
    if time_in_range(start_1, end_1, filtered_data['Time'].iloc[i])== True:
        time_labels.append(1)
    elif time_in_range(start_2, end_2, filtered_data['Time'].iloc[i])== True:
        time_labels.append(2)
    elif time_in_range(start_3, end_3, filtered_data['Time'].iloc[i])== True:
        time_labels.append(3)
    elif time_in_range(start_4, end_4, filtered_data['Time'].iloc[i])== True:
        time_labels.append(4)
    elif time_in_range(start_5, end_5, filtered_data['Time'].iloc[i])== True:
        time_labels.append(5)
    elif time_in_range(start_6, end_6, filtered_data['Time'].iloc[i])== True:
        time_labels.append(6)
    else:
        how.append(i)

# Print the time labels for verification
print(time_labels)

[3, 4, 3, 5, 3, 3, 5, 5, 5, 3, 4, 5, 4, 5, 3, 4, 5, 3, 5, 3, 4, 4, 5, 3, 3, 5, 4, 3, 4, 5, 4, 4, 2, 3, 3, 4, 3, 4, 3, 4, 4, 3, 5, 5, 5, 5, 4, 5, 4, 4, 3, 3, 4, 4, 3, 3, 5, 4, 4, 5, 3, 4, 3, 5, 3, 3, 4, 5, 3, 4, 3, 3, 3, 4, 3, 4, 3, 3, 4, 4, 5, 5, 3, 4, 4, 4, 4, 4, 5, 3, 3, 5, 5, 3, 3, 5, 3, 3, 3, 4, 4, 4, 3, 4, 3, 3, 4, 3, 5, 3, 3, 5, 5, 5, 4, 4, 5, 4, 5, 3, 3, 4, 4, 5, 4, 3, 3, 4, 5, 5, 3, 4, 4, 4, 3, 3, 3, 3, 4, 5, 5, 3, 3, 3, 4, 5, 4, 5, 5, 3, 3, 5, 5, 4, 5, 4, 4, 4, 4, 4, 5, 3, 4, 3, 5, 4, 4, 4, 4, 5, 5, 3, 4, 4, 3, 3, 5, 3, 4, 4, 4, 3, 5, 3, 4, 4, 3, 4, 5, 3, 4, 5, 4, 4, 5, 3, 3, 4, 5, 3, 4, 3, 5, 5, 3, 5, 4, 3, 5, 5, 3, 4, 5, 4, 5, 5, 4, 4, 5, 5, 3, 4, 5, 3, 4, 4, 5, 3, 5, 4, 5, 5, 3, 3, 5, 5, 3, 3, 4, 5, 3, 4, 4, 4, 5, 3, 5, 4, 3, 3, 4, 5, 3, 4, 5, 4, 5, 3, 5, 3, 3, 5, 3, 3, 4, 5, 3, 4, 5, 3, 4, 4, 4, 3, 4, 3, 3, 3, 5, 5, 4, 4, 4, 4, 4, 3, 3, 4, 5, 3, 3, 4, 5, 3, 4, 5, 5, 3, 3, 3, 5, 3, 5, 4, 4, 4, 4, 3, 4, 3, 4, 5, 3, 4, 5, 4, 3, 4, 3, 4, 3, 4, 3, 3, 4, 3, 4, 3, 4, 4, 5, 4, 4, 

In [76]:
time_labels2 = pd.DataFrame({
    'Time': [3, 4, 6, 6, 1, 5, 2, 3, 1, 3, 5, 5, 6, 5, 6, 3, 4, 5, 4, 5, 3, 4, 5, 6, 1, 2, 3, 5, 6, 1, 1, 2, 3, 4, 4, 5, 3, 3, 5, 4, 2, 3, 4, 5, 2, 4, 4, 5, 6, 3, 3, 4, 6, 6, 6, 4, 3, 1, 1, 1, 1, 2, 4, 4, 2, 3, 5, 1, 5, 5, 5, 1, 1, 4, 5, 2, 2, 6, 1, 2, 4, 6, 1, 2, 4, 3, 6, 2, 2, 3, 4, 4, 3, 3, 5, 6, 4, 2, 4, 5, 3, 4, 2, 3, 5, 1, 6, 3, 3, 6, 1, 1, 4, 5, 6, 3, 4, 2, 2, 2, 3, 3, 2, 3, 4, 6, 3, 4, 6, 1, 3, 4, 4, 5, 6, 1, 5, 6, 1, 3, 4, 4, 4, 1, 1, 4, 4, 1, 6, 1, 5, 2, 2, 2, 3, 3, 6, 5, 5, 6, 3, 3, 5, 3, 1, 2, 1, 3, 3, 4, 4, 4, 1, 3, 4, 3, 3, 4, 3, 5, 3, 3, 6, 2, 5, 5, 5, 6, 4, 4, 5, 6, 4, 5, 6, 3, 4, 4, 5, 4, 1, 3, 3, 1, 6, 6, 2, 4, 5, 6, 5, 6, 6, 2, 4, 2, 4, 4, 6, 1, 6, 2, 2, 1, 3, 3, 4, 5, 5, 2, 2, 3, 3, 3, 4, 5, 1, 1, 1, 2, 4, 5, 5, 1, 2, 2, 3, 3, 2, 6, 5, 5, 2, 4, 5, 1, 4, 4, 4, 4, 4, 5, 3, 6, 6, 4, 6, 6, 3, 6, 1, 5, 6, 6, 4, 4, 4, 4, 5, 2, 2, 5, 6, 4, 6, 4, 3, 3, 6, 5, 6, 3, 4, 4, 4, 3, 5, 3, 4, 4, 6, 4, 5, 1, 3, 4, 2, 5, 4, 6, 4, 5, 2, 3, 3, 4, 6, 1, 5, 1, 1, 3, 4, 1, 6, 5, 5, 3, 5, 1, 1, 1, 2, 2, 4, 1, 1, 2, 6, 3, 5, 1, 5, 1, 3, 4, 5, 1, 4, 5, 5, 4, 4, 5, 5, 6, 4, 5, 3, 4, 4, 5, 6, 2, 3, 5, 1, 2, 2, 1, 2, 2, 4, 5, 5, 6, 6, 2, 5, 2, 1, 5, 3, 3, 2, 4, 5, 6, 6, 2, 3, 6, 1, 4, 4, 6, 4, 2, 5, 1, 6, 3, 5, 4, 2, 2, 6, 3, 6, 2, 1, 4, 5, 6, 4, 5, 4, 5, 3, 5, 6, 1, 1, 6, 2, 3, 5, 6, 6, 6, 3, 3, 4, 5, 3, 4, 5, 6, 6, 6, 4, 2, 4, 4, 3, 4, 1, 3, 6, 1, 2, 6, 2, 3, 5, 6, 5, 4, 4, 4, 4, 4, 1, 6, 3, 3, 6, 1, 4, 5, 2, 3, 2, 6, 1, 3, 4, 6, 5, 6, 2, 3, 4, 5, 2, 5, 1, 2, 6, 3, 2, 3, 3, 1, 5, 1, 3, 5, 6, 4, 4, 4, 4, 3, 2, 4, 2, 3, 4, 5, 1, 2, 1, 6, 2, 3, 6, 4, 5, 4, 3, 4, 2, 3, 4, 6, 3, 4, 1, 3, 3, 6, 1, 2, 2, 4, 6, 4, 2, 3, 4, 6, 4, 1, 5, 4, 4, 4, 1, 2, 4, 5, 2, 3, 4, 6, 1, 5, 5, 5, 6, 2, 4, 4, 5, 3, 2, 4, 5, 4, 5, 4, 1, 2, 4, 5, 1, 4, 4, 6, 1, 1, 2, 2, 2, 1, 4, 5, 3, 4, 1, 5, 6, 3, 2, 3, 1, 4, 6, 1, 1, 1, 5, 6, 5, 3, 3, 5, 6, 1, 4, 4, 1, 2, 2, 3, 4, 1, 3, 4, 3, 4, 5, 5, 6, 1, 2, 2, 5, 5, 1, 2, 2, 4, 5, 3, 5, 3, 4, 3, 5, 4, 5, 1, 4, 4, 6, 4, 2, 2, 6, 1, 3, 5, 2, 3, 3, 6, 2, 3, 6, 2, 3, 4, 3, 1, 3, 3, 6, 2, 2, 4, 2, 6, 3, 1, 1, 4, 4, 4, 6, 6, 6, 4, 5, 1, 1, 5, 5, 1],})

In [77]:
data_truck_62222['Time'] = time_labels2
print(data_truck_62222)

           Date  Time                  Province  Loss Party Cause  \
0    01/01/2562     3                 Sing Buri     1    P6    C4   
1    01/01/2562     4               Suphan Buri     2    P5    C4   
2    01/01/2562     6  Phra Nakhon Si Ayutthaya     4    P9    C4   
3    02/01/2562     6                      Loei     3    P2    C4   
4    02/01/2562     1                    Ranong     1    P6    C6   
..          ...   ...                       ...   ...   ...   ...   
685  29/12/2562     1                    Roi Et     1    P4    C4   
686  29/12/2562     1                   Sisaket     2    P6    C4   
687  30/12/2562     5              Pathum Thani     1    P4    C4   
688  30/12/2562     5                Chiang Rai     2    P4    C4   
689  30/12/2562     1                       Nan     1    P4    C4   

    Main cause of accidents  
0                       M16  
1                       M15  
2                        M3  
3                        M2  
4                    

In [78]:
# ลบจุดทศนิยม
data_truck_62222['Time'] = data_truck_62222['Time'].astype(str)
data_truck_62222['Time'] = data_truck_62222['Time'].apply(lambda x: x.strip("0. "))

In [79]:
data_truck_62222

,Date,Time,Province,Loss,Party,Cause,Main cause of accidents
0,01/01/2562,3,Sing Buri,1,P6,C4,M16
1,01/01/2562,4,Suphan Buri,2,P5,C4,M15
2,01/01/2562,6,Phra Nakhon Si Ayutthaya,4,P9,C4,M3
3,02/01/2562,6,Loei,3,P2,C4,M2
4,02/01/2562,1,Ranong,1,P6,C6,M8
...,...,...,...,...,...,...,...
685,29/12/2562,1,Roi Et,1,P4,C4,M9
686,29/12/2562,1,Sisaket,2,P6,C4,M2
687,30/12/2562,5,Pathum Thani,1,P4,C4,M2
688,30/12/2562,5,Chiang Rai,2,P4,C4,M9


In [80]:
data_truck_62222 = data_truck_62222
data_truck_62222 = pd.DataFrame(data_truck_62222)
data_truck_62222['Time'] = data_truck_62222['Time'].apply(lambda x: x.strip())
data_truck_62222['Time'] = data_truck_62222['Time'].replace({'1': '16.01 - 20.00',
'2':'20.01 - 00.00',
'3':'00.01 - 04.00',
'4': '04.01 - 08.00',
'5': '08.01 - 12.00',
'6':'12.01 - 16.00'})

In [81]:
data_truck_62222.head()

,Date,Time,Province,Loss,Party,Cause,Main cause of accidents
0,01/01/2562,00.01 - 04.00,Sing Buri,1,P6,C4,M16
1,01/01/2562,04.01 - 08.00,Suphan Buri,2,P5,C4,M15
2,01/01/2562,12.01 - 16.00,Phra Nakhon Si Ayutthaya,4,P9,C4,M3
3,02/01/2562,12.01 - 16.00,Loei,3,P2,C4,M2
4,02/01/2562,16.01 - 20.00,Ranong,1,P6,C6,M8


# Data ที่ได้

### Data truck 2560

In [82]:
data_truck_600001.head()

,Date,Time,Province,Loss,Party,Cause,Main cause of accidents
0,02/01/2560,08.01 - 12.00,Khon Kaen,3,P8,C4,M4
1,04/01/2560,04.01 - 08.00,Phrae,2,P13,C4,M4
2,04/01/2560,20.01 - 00.00,Saraburi,2,P3,C3,M5
3,04/01/2560,20.01 - 00.00,Phetchaburi,2,P6,C6,M5
4,05/01/2560,04.01 - 08.00,Chiang Rai,2,P3,C3,M1


### Data truck 2561

In [83]:
data_truck_6111.head()

,Date,Time,Province,Loss,Party,Cause,Main cause of accidents
0,01/01/2561,20.01 - 00.00,Samut Sakhon,1,P6,C1,M16
1,01/01/2561,20.01 - 00.00,Rayong,1,P6,C1,M16
2,02/01/2561,08.01 - 12.00,Samut Sakhon,3,P1,C1,M19
3,03/01/2561,12.01 - 16.00,Surat Thani,2,P9,C1,M6
4,03/01/2561,16.01 - 20.00,Rayong,1,P6,C6,M1


In [84]:
data_truck_6111

,Date,Time,Province,Loss,Party,Cause,Main cause of accidents
0,01/01/2561,20.01 - 00.00,Samut Sakhon,1,P6,C1,M16
1,01/01/2561,20.01 - 00.00,Rayong,1,P6,C1,M16
2,02/01/2561,08.01 - 12.00,Samut Sakhon,3,P1,C1,M19
3,03/01/2561,12.01 - 16.00,Surat Thani,2,P9,C1,M6
4,03/01/2561,16.01 - 20.00,Rayong,1,P6,C6,M1
...,...,...,...,...,...,...,...
853,29/12/2561,08.01 - 12.00,Phra Nakhon Si Ayutthaya,4,P13,C1,M4
854,29/12/2561,08.01 - 12.00,Chanthaburi,2,P9,C1,M3
855,29/12/2561,08.01 - 12.00,Lamphun,2,P2,C1,M27
856,29/12/2561,12.01 - 16.00,Chachoengsao,3,P1,C1,M4


### Data truck 2562

In [85]:
data_truck_62222.head()

,Date,Time,Province,Loss,Party,Cause,Main cause of accidents
0,01/01/2562,00.01 - 04.00,Sing Buri,1,P6,C4,M16
1,01/01/2562,04.01 - 08.00,Suphan Buri,2,P5,C4,M15
2,01/01/2562,12.01 - 16.00,Phra Nakhon Si Ayutthaya,4,P9,C4,M3
3,02/01/2562,12.01 - 16.00,Loei,3,P2,C4,M2
4,02/01/2562,16.01 - 20.00,Ranong,1,P6,C6,M8


In [86]:
data_truck_62222['Cause']

0      C4
1      C4
2      C4
3      C4
4      C6
       ..
685    C4
686    C4
687    C4
688    C4
689    C4
Name: Cause, Length: 690, dtype: object

# Check Missing values

In [87]:
data_truck_600001.isnull().any()

Date                       False
Time                       False
Province                   False
Loss                       False
Party                      False
Cause                      False
Main cause of accidents    False
dtype: bool

In [88]:
data_truck_61.isnull().any()

Date                       False
Time                       False
Province                   False
Loss                       False
Party                      False
Cause                      False
Main cause of accidents    False
dtype: bool

In [89]:
data_truck_62222.isnull().any()

Date                       False
Time                       False
Province                   False
Loss                       False
Party                      False
Cause                      False
Main cause of accidents    False
dtype: bool

# Concat Data

In [90]:
data_concat = pd.concat([data_truck_600001,data_truck_6111,data_truck_62222])
data_concat

,Date,Time,Province,Loss,Party,Cause,Main cause of accidents
0,02/01/2560,08.01 - 12.00,Khon Kaen,3,P8,C4,M4
1,04/01/2560,04.01 - 08.00,Phrae,2,P13,C4,M4
2,04/01/2560,20.01 - 00.00,Saraburi,2,P3,C3,M5
3,04/01/2560,20.01 - 00.00,Phetchaburi,2,P6,C6,M5
4,05/01/2560,04.01 - 08.00,Chiang Rai,2,P3,C3,M1
...,...,...,...,...,...,...,...
685,29/12/2562,16.01 - 20.00,Roi Et,1,P4,C4,M9
686,29/12/2562,16.01 - 20.00,Sisaket,2,P6,C4,M2
687,30/12/2562,08.01 - 12.00,Pathum Thani,1,P4,C4,M2
688,30/12/2562,08.01 - 12.00,Chiang Rai,2,P4,C4,M9


In [91]:
data_concat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2430 entries, 0 to 689
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Date                     2430 non-null   object
 1   Time                     2430 non-null   object
 2   Province                 2430 non-null   object
 3   Loss                     2430 non-null   int64 
 4   Party                    2430 non-null   object
 5   Cause                    2430 non-null   object
 6   Main cause of accidents  2430 non-null   object
dtypes: int64(1), object(6)
memory usage: 151.9+ KB


In [92]:
set(data_concat['Party']) #Check เฉยๆ

{'P1', 'P13', 'P14', 'P15', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9'}

In [93]:
data_concat[(data_concat['Cause'] == 'C4' ) | (data_concat['Party'] == 'P4')]   #Check จำนวนดูเฉยๆ

,Date,Time,Province,Loss,Party,Cause,Main cause of accidents
0,02/01/2560,08.01 - 12.00,Khon Kaen,3,P8,C4,M4
1,04/01/2560,04.01 - 08.00,Phrae,2,P13,C4,M4
8,09/01/2560,12.01 - 16.00,Phetchaburi,4,P4,C15,M33
9,09/01/2560,16.01 - 20.00,Nakhon Sawan,4,P4,C4,M2
10,09/01/2560,20.01 - 00.00,Pathum Thani,1,P4,C4,M2
...,...,...,...,...,...,...,...
685,29/12/2562,16.01 - 20.00,Roi Et,1,P4,C4,M9
686,29/12/2562,16.01 - 20.00,Sisaket,2,P6,C4,M2
687,30/12/2562,08.01 - 12.00,Pathum Thani,1,P4,C4,M2
688,30/12/2562,08.01 - 12.00,Chiang Rai,2,P4,C4,M9


#เตรียมทำ Association

In [94]:
test_df = data_concat.copy() # copy ตารางข้างบนมา

In [95]:
test_df['for_association'] = test_df[['Party','Cause']].apply(list, axis=1) #รวมให้ party & cause

In [96]:
test_df.head()

,Date,Time,Province,Loss,Party,Cause,Main cause of accidents,for_association
0,02/01/2560,08.01 - 12.00,Khon Kaen,3,P8,C4,M4,"[P8, C4]"
1,04/01/2560,04.01 - 08.00,Phrae,2,P13,C4,M4,"[P13, C4]"
2,04/01/2560,20.01 - 00.00,Saraburi,2,P3,C3,M5,"[P3, C3]"
3,04/01/2560,20.01 - 00.00,Phetchaburi,2,P6,C6,M5,"[P6, C6]"
4,05/01/2560,04.01 - 08.00,Chiang Rai,2,P3,C3,M1,"[P3, C3]"


In [97]:
test_split = test_df.copy()   # เอาไว้ทำในช่วงเทศกาลสงกรานต์

In [98]:
test_split.head()

,Date,Time,Province,Loss,Party,Cause,Main cause of accidents,for_association
0,02/01/2560,08.01 - 12.00,Khon Kaen,3,P8,C4,M4,"[P8, C4]"
1,04/01/2560,04.01 - 08.00,Phrae,2,P13,C4,M4,"[P13, C4]"
2,04/01/2560,20.01 - 00.00,Saraburi,2,P3,C3,M5,"[P3, C3]"
3,04/01/2560,20.01 - 00.00,Phetchaburi,2,P6,C6,M5,"[P6, C6]"
4,05/01/2560,04.01 - 08.00,Chiang Rai,2,P3,C3,M1,"[P3, C3]"


In [99]:
test_split[['day', 'month', 'year']] = test_split['Date'].str.split('/', n=2, expand = True)  #แยกคอลัมน์ วันเดือนปีออกไว้ก่อน

In [100]:
test_split

,Date,Time,Province,Loss,Party,Cause,Main cause of accidents,for_association,day,month,year
0,02/01/2560,08.01 - 12.00,Khon Kaen,3,P8,C4,M4,"[P8, C4]",02,01,2560
1,04/01/2560,04.01 - 08.00,Phrae,2,P13,C4,M4,"[P13, C4]",04,01,2560
2,04/01/2560,20.01 - 00.00,Saraburi,2,P3,C3,M5,"[P3, C3]",04,01,2560
3,04/01/2560,20.01 - 00.00,Phetchaburi,2,P6,C6,M5,"[P6, C6]",04,01,2560
4,05/01/2560,04.01 - 08.00,Chiang Rai,2,P3,C3,M1,"[P3, C3]",05,01,2560
...,...,...,...,...,...,...,...,...,...,...,...
685,29/12/2562,16.01 - 20.00,Roi Et,1,P4,C4,M9,"[P4, C4]",29,12,2562
686,29/12/2562,16.01 - 20.00,Sisaket,2,P6,C4,M2,"[P6, C4]",29,12,2562
687,30/12/2562,08.01 - 12.00,Pathum Thani,1,P4,C4,M2,"[P4, C4]",30,12,2562
688,30/12/2562,08.01 - 12.00,Chiang Rai,2,P4,C4,M9,"[P4, C4]",30,12,2562


In [101]:
test_split.isnull().any()

Date                       False
Time                       False
Province                   False
Loss                       False
Party                      False
Cause                      False
Main cause of accidents    False
for_association            False
day                        False
month                      False
year                       False
dtype: bool

# ทำเป็นตารางเอาเฉพาะคอลัมน์ที่ต้องการมา

In [102]:
data_as1 = test_split[['Date','Time','Province','Loss','Party','Cause','Main cause of accidents']].replace(0,np.nan)

In [103]:
data_as1.head()

,Date,Time,Province,Loss,Party,Cause,Main cause of accidents
0,02/01/2560,08.01 - 12.00,Khon Kaen,3,P8,C4,M4
1,04/01/2560,04.01 - 08.00,Phrae,2,P13,C4,M4
2,04/01/2560,20.01 - 00.00,Saraburi,2,P3,C3,M5
3,04/01/2560,20.01 - 00.00,Phetchaburi,2,P6,C6,M5
4,05/01/2560,04.01 - 08.00,Chiang Rai,2,P3,C3,M1


# ทำ Association

## ต้องการหาว่าแต่ละช่วงเวลาในการเกิดอุบัติเหตุ มักมีรถคู่กรณีกับรถต้นเหตุแบบใด

#### ในช่วงเวลา 16.01 - 20.00 มักมีรถคู่กรณีกับรถต้นเหตุแบบใด

#### รถคู่กรณีและรถต้นเหตุที่พบในช่วงเวลา  16.01 - 20.00

In [104]:
data_as2 = test_split[['Time','for_association']].replace('-',np.nan)

In [105]:
data_as2

,Time,for_association
0,08.01 - 12.00,"[P8, C4]"
1,04.01 - 08.00,"[P13, C4]"
2,20.01 - 00.00,"[P3, C3]"
3,20.01 - 00.00,"[P6, C6]"
4,04.01 - 08.00,"[P3, C3]"
...,...,...
685,16.01 - 20.00,"[P4, C4]"
686,16.01 - 20.00,"[P6, C4]"
687,08.01 - 12.00,"[P4, C4]"
688,08.01 - 12.00,"[P4, C4]"


In [106]:
data_as2.groupby(['Time']).sum()

,for_association
Time,
00.01 - 04.00,"[P3, C3, P5, C5, P4, C4, P2, C2, P6, C6, P13, ..."
04.01 - 08.00,"[P13, C4, P3, C3, P2, C2, P4, C4, P4, C4, P4, ..."
08.01 - 12.00,"[P8, C4, P6, C6, P4, C4, P4, C4, P13, C4, P6, ..."
12.01 - 16.00,"[P4, C15, P4, C4, P4, C4, P6, C6, P13, C4, P2,..."
16.01 - 20.00,"[P4, C4, P4, C4, P6, C6, P5, C4, P13, C4, P1, ..."
20.01 - 00.00,"[P3, C3, P6, C6, P4, C4, P2, C2, P4, C4, P4, C..."
nan,"[P6, C6, P6, C1, P6, C1]"


In [107]:
data_as2.groupby('Time')['for_association'].apply(list)

Time
00.01 - 04.00    [[P3, C3], [P5, C5], [P4, C4], [P2, C2], [P6, ...
04.01 - 08.00    [[P13, C4], [P3, C3], [P2, C2], [P4, C4], [P4,...
08.01 - 12.00    [[P8, C4], [P6, C6], [P4, C4], [P4, C4], [P13,...
12.01 - 16.00    [[P4, C15], [P4, C4], [P4, C4], [P6, C6], [P13...
16.01 - 20.00    [[P4, C4], [P4, C4], [P6, C6], [P5, C4], [P13,...
20.01 - 00.00    [[P3, C3], [P6, C6], [P4, C4], [P2, C2], [P4, ...
nan                                 [[P6, C6], [P6, C1], [P6, C1]]
Name: for_association, dtype: object

In [108]:
# ทำให้ Data มีรรูปแบบเป็น transaction

data_asso_time1 = data_as2.groupby(['Time'])['for_association'].apply(list).reset_index()
data_asso_time1_trans = data_asso_time1['for_association'].apply(lambda x: [str(item) for item in x])
data_asso_time1_trans

0    [['P3', 'C3'], ['P5', 'C5'], ['P4', 'C4'], ['P...
1    [['P13', 'C4'], ['P3', 'C3'], ['P2', 'C2'], ['...
2    [['P8', 'C4'], ['P6', 'C6'], ['P4', 'C4'], ['P...
3    [['P4', 'C15'], ['P4', 'C4'], ['P4', 'C4'], ['...
4    [['P4', 'C4'], ['P4', 'C4'], ['P6', 'C6'], ['P...
5    [['P3', 'C3'], ['P6', 'C6'], ['P4', 'C4'], ['P...
6           [['P6', 'C6'], ['P6', 'C1'], ['P6', 'C1']]
Name: for_association, dtype: object

In [109]:
min_sup = 0.87

sup_data_asso_time1_trans = list(apriori(data_asso_time1_trans, min_support = min_sup))
for i in sup_data_asso_time1_trans:
    if i[0].__len__()>1:
        print('yes')
        print(i)

yes
RelationRecord(items=frozenset({"['P6', 'C6']", "['P6', 'C1']"}), support=1.0, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({"['P6', 'C6']", "['P6', 'C1']"}), confidence=1.0, lift=1.0), OrderedStatistic(items_base=frozenset({"['P6', 'C1']"}), items_add=frozenset({"['P6', 'C6']"}), confidence=1.0, lift=1.0), OrderedStatistic(items_base=frozenset({"['P6', 'C6']"}), items_add=frozenset({"['P6', 'C1']"}), confidence=1.0, lift=1.0)])


In [110]:
min_sup = 0.87
min_lifts = 0.8
sup_data_morning = list(apriori(data_asso_time1_trans, min_support = min_sup, min_lift=min_lifts))
for i in sup_data_morning:
    if i[0].__len__()>1:
        print('yes')
        print(i)

yes
RelationRecord(items=frozenset({"['P6', 'C6']", "['P6', 'C1']"}), support=1.0, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({"['P6', 'C6']", "['P6', 'C1']"}), confidence=1.0, lift=1.0), OrderedStatistic(items_base=frozenset({"['P6', 'C1']"}), items_add=frozenset({"['P6', 'C6']"}), confidence=1.0, lift=1.0), OrderedStatistic(items_base=frozenset({"['P6', 'C6']"}), items_add=frozenset({"['P6', 'C1']"}), confidence=1.0, lift=1.0)])


#### ในช่วงเวลา 20.01 - 00.00 มักมีรถคู่กรณีกับรถต้นเหตุแบบใด

#### ในช่วงเวลา00.01 - 04.00 มักมีรถคู่กรณีกับรถต้นเหตุแบบใด

#### ในช่วงเวลา 04.01 - 08.00 มักมีรถคู่กรณีกับรถต้นเหตุแบบใด

#### ในช่วงเวลา 08.01 - 12.00 แต่ละจังหวัด มักมีรถคู่กรณีกับรถต้นเหตุแบบใด

#### ในช่วงเวลา 12.01 - 16.00 แต่ละจังหวัด มักมีรถคู่กรณีกับรถต้นเหตุแบบใด

#### สาเหตุการเกิด กับรถคู่กรณีและรถต้นเหตุ

In [111]:
data_as3 = test_split[['Main cause of accidents','for_association']].replace('-',np.nan)

In [112]:
data_as3

,Main cause of accidents,for_association
0,M4,"[P8, C4]"
1,M4,"[P13, C4]"
2,M5,"[P3, C3]"
3,M5,"[P6, C6]"
4,M1,"[P3, C3]"
...,...,...
685,M9,"[P4, C4]"
686,M2,"[P6, C4]"
687,M2,"[P4, C4]"
688,M9,"[P4, C4]"


In [113]:
data_as3.groupby(['Main cause of accidents']).sum()

,for_association
Main cause of accidents,
M1,"[P3, C3, P3, C3, P4, C4, P4, C4, P4, C4, P4, C..."
M10,"[P5, C5, P8, C4, P6, C4, P4, C4, P4, C4, P6, C..."
M11,"[P1, C1, P13, C4, P2, C2, P4, C4, P1, C1, P2, ..."
M12,"[P1, C1, P1, C1, P6, C6, P6, C6, P6, C6, P9, C..."
M14,"[P9, C4, P6, C6, P1, C1, P4, C4, P1, C1, P13, ..."
M15,"[P1, C4, P6, C6, P1, C1, P2, C4, P2, C2, P6, C..."
M16,"[P13, C4, P1, C4, P1, C4, P6, C4, P1, C4, P1, ..."
M17,"[P4, C4, P6, C4, P4, C4, P6, C4, P1, C4, P6, C..."
M19,"[P6, C6, P13, C4, P4, C4, P2, C2, P2, C2, P13,..."


In [114]:
data_as3.groupby('Main cause of accidents')['for_association'].apply(list)

Main cause of accidents
M1                    [[P3, C3], [P3, C3], [P4, C4], [P4, C4], [P4, ...
M10                   [[P5, C5], [P8, C4], [P6, C4], [P4, C4], [P4, ...
M11                   [[P1, C1], [P13, C4], [P2, C2], [P4, C4], [P1,...
M12                   [[P1, C1], [P1, C1], [P6, C6], [P6, C6], [P6, ...
M14                   [[P9, C4], [P6, C6], [P1, C1], [P4, C4], [P1, ...
M15                   [[P1, C4], [P6, C6], [P1, C1], [P2, C4], [P2, ...
M16                   [[P13, C4], [P1, C4], [P1, C4], [P6, C4], [P1,...
M17                   [[P4, C4], [P6, C4], [P4, C4], [P6, C4], [P1, ...
M19                   [[P6, C6], [P13, C4], [P4, C4], [P2, C2], [P2,...
M2                    [[P4, C4], [P4, C4], [P2, C2], [P4, C4], [P2, ...
M20                   [[P4, C4], [P4, C4], [P2, C2], [P13, C4], [P9,...
M21                                                         [[P13, C4]]
M22                   [[P13, C4], [P4, C4], [P13, C4], [P13, C4], [P...
M23                                   [[

In [115]:
# ทำให้ Data มีรรูปแบบเป็น transaction

data_asso_Main1 = data_as3.groupby(['Main cause of accidents'])['for_association'].apply(list).reset_index()
data_asso_Main1_trans = data_asso_Main1['for_association'].apply(lambda x: [str(item) for item in x])
data_asso_Main1_trans

0     [['P3', 'C3'], ['P3', 'C3'], ['P4', 'C4'], ['P...
1     [['P5', 'C5'], ['P8', 'C4'], ['P6', 'C4'], ['P...
2     [['P1', 'C1'], ['P13', 'C4'], ['P2', 'C2'], ['...
3     [['P1', 'C1'], ['P1', 'C1'], ['P6', 'C6'], ['P...
4     [['P9', 'C4'], ['P6', 'C6'], ['P1', 'C1'], ['P...
5     [['P1', 'C4'], ['P6', 'C6'], ['P1', 'C1'], ['P...
6     [['P13', 'C4'], ['P1', 'C4'], ['P1', 'C4'], ['...
7     [['P4', 'C4'], ['P6', 'C4'], ['P4', 'C4'], ['P...
8     [['P6', 'C6'], ['P13', 'C4'], ['P4', 'C4'], ['...
9     [['P4', 'C4'], ['P4', 'C4'], ['P2', 'C2'], ['P...
10    [['P4', 'C4'], ['P4', 'C4'], ['P2', 'C2'], ['P...
11                                      [['P13', 'C4']]
12    [['P13', 'C4'], ['P4', 'C4'], ['P13', 'C4'], [...
13        [['P13', 'C4'], ['P13', 'C1'], ['P13', 'C4']]
14                      [['P13', 'C1'], ['P13', 'C13']]
15    [['P1', 'C4'], ['P13', 'C4'], ['P13', 'C4'], [...
16    [['P4', 'C4'], ['P6', 'C6'], ['P4', 'C4'], ['P...
17    [['P2', 'C2'], ['P2', 'C2'], ['P1', 'C1'],

In [116]:
min_sup = 0.57

sup_data_asso_Main1_trans = list(apriori(data_asso_Main1_trans, min_support = min_sup))
for i in sup_data_asso_Main1_trans:
    if i[0].__len__()>1:
        print('yes')
        print(i)

yes
RelationRecord(items=frozenset({"['P1', 'C1']", "['P4', 'C4']"}), support=0.625, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({"['P1', 'C1']", "['P4', 'C4']"}), confidence=0.625, lift=1.0), OrderedStatistic(items_base=frozenset({"['P1', 'C1']"}), items_add=frozenset({"['P4', 'C4']"}), confidence=0.9090909090909091, lift=1.1636363636363636), OrderedStatistic(items_base=frozenset({"['P4', 'C4']"}), items_add=frozenset({"['P1', 'C1']"}), confidence=0.8, lift=1.1636363636363638)])
yes
RelationRecord(items=frozenset({"['P4', 'C4']", "['P13', 'C1']"}), support=0.59375, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({"['P4', 'C4']", "['P13', 'C1']"}), confidence=0.59375, lift=1.0), OrderedStatistic(items_base=frozenset({"['P13', 'C1']"}), items_add=frozenset({"['P4', 'C4']"}), confidence=0.9047619047619048, lift=1.158095238095238), OrderedStatistic(items_base=frozenset({"['P4', 'C4']"}), items_add=frozenset({"['P13', 'C

#### สาเหตุการเกิด กับรถคู่กรณีและรถต้นเหตุ

In [117]:
data_as3 = test_split[['Main cause of accidents','for_association']].replace('-',np.nan)

In [118]:
data_as3

,Main cause of accidents,for_association
0,M4,"[P8, C4]"
1,M4,"[P13, C4]"
2,M5,"[P3, C3]"
3,M5,"[P6, C6]"
4,M1,"[P3, C3]"
...,...,...
685,M9,"[P4, C4]"
686,M2,"[P6, C4]"
687,M2,"[P4, C4]"
688,M9,"[P4, C4]"


In [119]:
data_as3.groupby(['Main cause of accidents']).sum()

,for_association
Main cause of accidents,
M1,"[P3, C3, P3, C3, P4, C4, P4, C4, P4, C4, P4, C..."
M10,"[P5, C5, P8, C4, P6, C4, P4, C4, P4, C4, P6, C..."
M11,"[P1, C1, P13, C4, P2, C2, P4, C4, P1, C1, P2, ..."
M12,"[P1, C1, P1, C1, P6, C6, P6, C6, P6, C6, P9, C..."
M14,"[P9, C4, P6, C6, P1, C1, P4, C4, P1, C1, P13, ..."
M15,"[P1, C4, P6, C6, P1, C1, P2, C4, P2, C2, P6, C..."
M16,"[P13, C4, P1, C4, P1, C4, P6, C4, P1, C4, P1, ..."
M17,"[P4, C4, P6, C4, P4, C4, P6, C4, P1, C4, P6, C..."
M19,"[P6, C6, P13, C4, P4, C4, P2, C2, P2, C2, P13,..."


In [120]:
data_as3.groupby('Main cause of accidents')['for_association'].apply(list)

Main cause of accidents
M1                    [[P3, C3], [P3, C3], [P4, C4], [P4, C4], [P4, ...
M10                   [[P5, C5], [P8, C4], [P6, C4], [P4, C4], [P4, ...
M11                   [[P1, C1], [P13, C4], [P2, C2], [P4, C4], [P1,...
M12                   [[P1, C1], [P1, C1], [P6, C6], [P6, C6], [P6, ...
M14                   [[P9, C4], [P6, C6], [P1, C1], [P4, C4], [P1, ...
M15                   [[P1, C4], [P6, C6], [P1, C1], [P2, C4], [P2, ...
M16                   [[P13, C4], [P1, C4], [P1, C4], [P6, C4], [P1,...
M17                   [[P4, C4], [P6, C4], [P4, C4], [P6, C4], [P1, ...
M19                   [[P6, C6], [P13, C4], [P4, C4], [P2, C2], [P2,...
M2                    [[P4, C4], [P4, C4], [P2, C2], [P4, C4], [P2, ...
M20                   [[P4, C4], [P4, C4], [P2, C2], [P13, C4], [P9,...
M21                                                         [[P13, C4]]
M22                   [[P13, C4], [P4, C4], [P13, C4], [P13, C4], [P...
M23                                   [[

In [121]:
# ทำให้ Data มีรรูปแบบเป็น transaction

data_asso_Main1 = data_as3.groupby(['Main cause of accidents'])['for_association'].apply(list).reset_index()
data_asso_Main1_trans = data_asso_Main1['for_association'].apply(lambda x: [str(item) for item in x])
data_asso_Main1_trans

0     [['P3', 'C3'], ['P3', 'C3'], ['P4', 'C4'], ['P...
1     [['P5', 'C5'], ['P8', 'C4'], ['P6', 'C4'], ['P...
2     [['P1', 'C1'], ['P13', 'C4'], ['P2', 'C2'], ['...
3     [['P1', 'C1'], ['P1', 'C1'], ['P6', 'C6'], ['P...
4     [['P9', 'C4'], ['P6', 'C6'], ['P1', 'C1'], ['P...
5     [['P1', 'C4'], ['P6', 'C6'], ['P1', 'C1'], ['P...
6     [['P13', 'C4'], ['P1', 'C4'], ['P1', 'C4'], ['...
7     [['P4', 'C4'], ['P6', 'C4'], ['P4', 'C4'], ['P...
8     [['P6', 'C6'], ['P13', 'C4'], ['P4', 'C4'], ['...
9     [['P4', 'C4'], ['P4', 'C4'], ['P2', 'C2'], ['P...
10    [['P4', 'C4'], ['P4', 'C4'], ['P2', 'C2'], ['P...
11                                      [['P13', 'C4']]
12    [['P13', 'C4'], ['P4', 'C4'], ['P13', 'C4'], [...
13        [['P13', 'C4'], ['P13', 'C1'], ['P13', 'C4']]
14                      [['P13', 'C1'], ['P13', 'C13']]
15    [['P1', 'C4'], ['P13', 'C4'], ['P13', 'C4'], [...
16    [['P4', 'C4'], ['P6', 'C6'], ['P4', 'C4'], ['P...
17    [['P2', 'C2'], ['P2', 'C2'], ['P1', 'C1'],

In [122]:
min_sup = 0.57

sup_data_asso_Main1_trans = list(apriori(data_asso_Main1_trans, min_support = min_sup))
for i in sup_data_asso_Main1_trans:
    if i[0].__len__()>1:
        print('yes')
        print(i)

yes
RelationRecord(items=frozenset({"['P1', 'C1']", "['P4', 'C4']"}), support=0.625, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({"['P1', 'C1']", "['P4', 'C4']"}), confidence=0.625, lift=1.0), OrderedStatistic(items_base=frozenset({"['P1', 'C1']"}), items_add=frozenset({"['P4', 'C4']"}), confidence=0.9090909090909091, lift=1.1636363636363636), OrderedStatistic(items_base=frozenset({"['P4', 'C4']"}), items_add=frozenset({"['P1', 'C1']"}), confidence=0.8, lift=1.1636363636363638)])
yes
RelationRecord(items=frozenset({"['P4', 'C4']", "['P13', 'C1']"}), support=0.59375, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({"['P4', 'C4']", "['P13', 'C1']"}), confidence=0.59375, lift=1.0), OrderedStatistic(items_base=frozenset({"['P13', 'C1']"}), items_add=frozenset({"['P4', 'C4']"}), confidence=0.9047619047619048, lift=1.158095238095238), OrderedStatistic(items_base=frozenset({"['P4', 'C4']"}), items_add=frozenset({"['P13', 'C

#Classification

##เตรียมข้อมูลทำ Decision-Tree

In [123]:
data_truck_num = data_concat
data_truck_num = pd.DataFrame(data_concat)
data_truck_num['Time'] = data_truck_num['Time'].apply(lambda x: x.strip())
data_truck_num['Time'] = data_truck_num['Time'].replace({'16.01 - 20.00': '1',
'20.01 - 00.00':'2',
'00.01 - 04.00':'3',
'04.01 - 08.00': '4',
'08.01 - 12.00': '5',
'12.01 - 16.00':'6'})

In [124]:
data_truck_num

,Date,Time,Province,Loss,Party,Cause,Main cause of accidents
0,02/01/2560,5,Khon Kaen,3,P8,C4,M4
1,04/01/2560,4,Phrae,2,P13,C4,M4
2,04/01/2560,2,Saraburi,2,P3,C3,M5
3,04/01/2560,2,Phetchaburi,2,P6,C6,M5
4,05/01/2560,4,Chiang Rai,2,P3,C3,M1
...,...,...,...,...,...,...,...
685,29/12/2562,1,Roi Et,1,P4,C4,M9
686,29/12/2562,1,Sisaket,2,P6,C4,M2
687,30/12/2562,5,Pathum Thani,1,P4,C4,M2
688,30/12/2562,5,Chiang Rai,2,P4,C4,M9


In [125]:
data_truck_num = data_concat
datamaincity = pd.DataFrame(data_concat)
datamaincity['Province'] = datamaincity['Province'].apply(lambda x: x.strip())
datamaincity['Province'] = datamaincity['Province'].replace({
    'Chiang Mai': 1,
    'Chiang Rai': 2,
    'Lampang': 3,
    'Lamphun': 4,
    'Mae Hong Son': 5,
    'Nan': 6,
    'Phayao': 7,
    'Phrae': 8,
    'Uttaradit': 9,
    'Kalasin': 10,
    'Khon Kaen': 11,
    'Chaiyaphum': 12,
    'Nakhon Phanom': 13,
    'Nakhon Ratchasima': 14,
    'Bueng Kan': 15,
    'Buriram': 16,
    'Maha Sarakham': 17,
    'Mukdahan': 18,
    'Yasothon': 19,
    'Roi Et': 20,
    'Loei': 21,
    'Sakon Nakhon': 22,
    'Surin': 23,
    'Sisaket': 24,
    'Nong Khai': 25,
    'Nong Bua Lamphu': 26,
    'Udon Thani': 27,
    'Ubon Ratchathani': 28,
    'Amnat Charoen': 29,
    'Bangkok': 30,
    'Kamphaeng Phet': 31,
    'Chai Nat': 32,
    'Nakhon Nayok': 33,
    'Nakhon Pathom': 34,
    'Nakhon Sawan': 35,
    'Nonthaburi': 36,
    'Pathum Thani': 37,
    'Phra Nakhon Si Ayutthaya': 38,
    'Phichit': 39,
    'Phitsanulok': 40,
    'Phetchabun': 41,
    'Lopburi': 42,
    'Samut Prakan': 43,
    'Samut Songkhram': 44,
    'Samut Sakhon': 45,
    'Sing Buri': 46,
    'Sukhothai': 47,
    'Suphan Buri': 48,
    'Saraburi': 49,
    'Ang Thong': 50,
    'Uthai Thani': 51,
    'Chanthaburi': 52,
    'Chachoengsao': 53,
    'Chonburi': 54,
    'Trat': 55,
    'Prachinburi': 56,
    'Rayong': 57,
    'Sa Kaeo': 58,
    'Kanchanaburi': 59,
    'Tak': 60,
    'Prachuap Khiri Khan': 61,
    'Phetchaburi': 62,
    'Ratchaburi': 63,
    'Krabi': 64,
    'Chumphon': 65,
    'Trang': 66,
    'Nakhon Si Thammarat': 67,
    'Narathiwat': 68,
    'Pattani': 69,
    'Phang Nga': 70,
    'Phatthalung': 71,
    'Phuket': 72,
    'Ranong': 73,
    'Satun': 74,
    'Songkhla': 75,
    'Surat Thani': 76,
    'Yala': 77
})

In [126]:
#เปลี่ยนสาเหตุเป็นตัวเลขที่กำหนดไว้
data_truck_num = data_concat
datamain = pd.DataFrame(data_concat)
datamain['Main cause of accidents'] = datamain['Main cause of accidents'].apply(lambda x: x.strip())
datamain['Main cause of accidents'] = datamain['Main cause of accidents'].replace({ 'M1': 1,
    'M2': 2,
    'M3': 3,
    'M4': 4,
    'M5': 5,
    'M6': 6,
    'M7': 7,
    'M8': 8,
    'M9': 9,
    'M10': 10,
    'M11': 11,
    'M12': 12,
    'M13': 13,
    'M14': 14,
    'M15': 15,
    'M16': 16,
    'M17': 17,
    'M18': 18,
    'M19': 19,
    'M20': 20,
    'M21': 21,
    'M22': 22,
    'M23': 23,
    'M24': 24,
    'M25': 25,
    'M26': 26,
    'M27': 27,
    'M28': 28,
    'M29': 29,
    'M30': 30,
    'M31': 31,
    'M32': 32,
    'M33': 33,
    'M34': 34
})

In [127]:
# เปลี่ยนตัวแปรคู่กรณี (Party) เป็นตัวเลขที่กำหนดไว้
data_truck_num = data_concat
datapar = pd.DataFrame(data_concat)
datapar['Party'] = datapar['Party'].apply(lambda x: x.strip())
datapar['Party'] = datapar['Party'].replace({  'P1': 1,
    'P2': 2,
    'P3': 3,
    'P4': 4,
    'P5': 5,
    'P6': 6,
    'P7': 7,
    'P8': 8,
    'P9': 9,
    'P10': 10,
    'P11': 11,
    'P12': 12,
    'P13': 13,
    'P14': 14,
    'P15': 15})


In [128]:
# เปลี่ยนตัวแปรรถต้นเหตุ (Cause) เป็นตัวเลขที่กำหนดไว้
data_truck_num = data_concat
datacau = pd.DataFrame(data_concat)
datacau['Cause'] = datacau['Cause'].apply(lambda x: x.strip())
datacau['Cause'] = datacau['Cause'].replace({ 'C1': 1,
    'C2': 2,
    'C3': 3,
    'C4': 4,
    'C5': 5,
    'C6': 6,
    'C7': 7,
    'C8': 8,
    'C9': 9,
    'C10': 10,
    'C11': 11,
    'C12': 12,
    'C13': 13,
    'C14': 14,
    'C15': 15
})

In [129]:
type(data_truck_num['Date'][0])

pandas.core.series.Series

In [130]:
data_truck_num.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2430 entries, 0 to 689
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Date                     2430 non-null   object
 1   Time                     2430 non-null   object
 2   Province                 2430 non-null   object
 3   Loss                     2430 non-null   int64 
 4   Party                    2430 non-null   int64 
 5   Cause                    2430 non-null   int64 
 6   Main cause of accidents  2430 non-null   object
dtypes: int64(3), object(4)
memory usage: 216.4+ KB


In [131]:
DATA_truck_df_for_tree = data_truck_num.copy()

In [132]:
DATA_truck_df_for_tree

,Date,Time,Province,Loss,Party,Cause,Main cause of accidents
0,02/01/2560,5,11,3,8,4,4
1,04/01/2560,4,8,2,13,4,4
2,04/01/2560,2,49,2,3,3,5
3,04/01/2560,2,62,2,6,6,5
4,05/01/2560,4,2,2,3,3,1
...,...,...,...,...,...,...,...
685,29/12/2562,1,20,1,4,4,9
686,29/12/2562,1,24,2,6,4,2
687,30/12/2562,5,37,1,4,4,2
688,30/12/2562,5,2,2,4,4,9


###เเยกวันเดือนปีออกจากเวลา
เเยกวันเดือนปี
ออกจากเวลา ในคอลัมน์ InvoiceDate ด้วย split

In [133]:
DATA_truck_df_for_tree['Date'].str.split().iloc[0][0]

'02/01/2560'

In [134]:
date = []
for i in range(len(DATA_truck_df_for_tree['Date'])):
    x = DATA_truck_df_for_tree['Date'].iloc[i].split()[0]
    date.append(x)

In [135]:
date[0].split('/')

['02', '01', '2560']

ปรับ 'date' เป็นสตริงที่มีรูปแบบ 'dd/mm/yyyy' (วัน/เดือน/ปี)

In [136]:
day =[]
month =[]
year =[]
for i in range(len(DATA_truck_df_for_tree['Date'])):
  x = date[i].split('/')[0]
  y = date[i].split('/')[1]
  z = date[i].split('/')[2]
  month.append(x)
  day.append(y)
  year.append(z)

สร้าง DataFrame ใหม่ ตามข้อมูลที่มีอยู่เเละเพิ่มคอลัมน์ใหม่ ('date', 'day', 'month', 'year')

In [137]:
DATA_truck_df_New_tree = data_concat.copy()

In [138]:
DATA_truck_df_New_tree['date']=date
DATA_truck_df_New_tree['day']=month
DATA_truck_df_New_tree['month']=day
DATA_truck_df_New_tree['year']=year

In [139]:
DATA_truck_df_New_tree

,Date,Time,Province,Loss,Party,Cause,Main cause of accidents,date,day,month,year
0,02/01/2560,5,11,3,8,4,4,02/01/2560,02,01,2560
1,04/01/2560,4,8,2,13,4,4,04/01/2560,04,01,2560
2,04/01/2560,2,49,2,3,3,5,04/01/2560,04,01,2560
3,04/01/2560,2,62,2,6,6,5,04/01/2560,04,01,2560
4,05/01/2560,4,2,2,3,3,1,05/01/2560,05,01,2560
...,...,...,...,...,...,...,...,...,...,...,...
685,29/12/2562,1,20,1,4,4,9,29/12/2562,29,12,2562
686,29/12/2562,1,24,2,6,4,2,29/12/2562,29,12,2562
687,30/12/2562,5,37,1,4,4,2,30/12/2562,30,12,2562
688,30/12/2562,5,2,2,4,4,9,30/12/2562,30,12,2562


In [140]:
type(DATA_truck_df_New_tree['day'][0])

pandas.core.series.Series

In [141]:
DATA_truck_df_New_tree.isnull().any()

Date                       False
Time                       False
Province                   False
Loss                       False
Party                      False
Cause                      False
Main cause of accidents    False
date                       False
day                        False
month                      False
year                       False
dtype: bool

In [142]:
DATA_truck_df_New_tree

,Date,Time,Province,Loss,Party,Cause,Main cause of accidents,date,day,month,year
0,02/01/2560,5,11,3,8,4,4,02/01/2560,02,01,2560
1,04/01/2560,4,8,2,13,4,4,04/01/2560,04,01,2560
2,04/01/2560,2,49,2,3,3,5,04/01/2560,04,01,2560
3,04/01/2560,2,62,2,6,6,5,04/01/2560,04,01,2560
4,05/01/2560,4,2,2,3,3,1,05/01/2560,05,01,2560
...,...,...,...,...,...,...,...,...,...,...,...
685,29/12/2562,1,20,1,4,4,9,29/12/2562,29,12,2562
686,29/12/2562,1,24,2,6,4,2,29/12/2562,29,12,2562
687,30/12/2562,5,37,1,4,4,2,30/12/2562,30,12,2562
688,30/12/2562,5,2,2,4,4,9,30/12/2562,30,12,2562


In [143]:
data_tree1 = DATA_truck_df_New_tree[['day','month','year','Time','Province','Loss','Party','Cause','Main cause of accidents']].replace('0',np.nan)

In [144]:
data_tree1

,day,month,year,Time,Province,Loss,Party,Cause,Main cause of accidents
0,02,01,2560,5,11,3,8,4,4
1,04,01,2560,4,8,2,13,4,4
2,04,01,2560,2,49,2,3,3,5
3,04,01,2560,2,62,2,6,6,5
4,05,01,2560,4,2,2,3,3,1
...,...,...,...,...,...,...,...,...,...
685,29,12,2562,1,20,1,4,4,9
686,29,12,2562,1,24,2,6,4,2
687,30,12,2562,5,37,1,4,4,2
688,30,12,2562,5,2,2,4,4,9


In [145]:
data_tree1['day'] = data_tree1['day'].astype(bool)
data_tree1['month'] = data_tree1['month'].astype(bool)
data_tree1['year'] = data_tree1['year'].astype(bool)
data_tree1['Time'] = data_tree1['Time'].astype(bool)
data_tree1['Province'] = data_tree1['Province'].astype(bool)
data_tree1['Loss'] = data_tree1['Loss'].astype(bool)
data_tree1['Party'] = data_tree1['Party'].astype(bool)
data_tree1['Cause'] = data_tree1['Cause'].astype(bool)
data_tree1['Main cause of accidents'] = data_tree1['Main cause of accidents'].astype(bool)

In [146]:
data_tree1.isnull().any()

day                        False
month                      False
year                       False
Time                       False
Province                   False
Loss                       False
Party                      False
Cause                      False
Main cause of accidents    False
dtype: bool

In [147]:
X = data_tree1.values[:,:-1]
Y = data_tree1.values[:,-1]

In [148]:
X

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

In [149]:
Y

array([ True,  True,  True, ...,  True,  True,  True])

In [150]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='mean')  # or 'median', 'constant', etc.
X_imputed = imputer.fit_transform(X)

In [154]:
x_train, x_test, y_train, y_test = train_test_split(X_imputed, Y, test_size= 0.3, random_state= 10)

In [155]:
print("Shape of x_train:", x_train.shape)
print("Shape of x_test:", x_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

Shape of x_train: (1701, 8)
Shape of x_test: (729, 8)
Shape of y_train: (1701,)
Shape of y_test: (729,)


In [156]:
y_train

array([ True,  True,  True, ...,  True,  True,  True])

In [157]:
clf_entropy = DecisionTreeClassifier (criterion = "entropy", random_state= 100,max_depth=3, min_samples_leaf=4)
clf_entropy.fit (x_train, y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=3, min_samples_leaf=4,
                       random_state=100)

In [158]:
y_pred_en = clf_entropy.predict (x_test)
y_pred_en

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [160]:
print("Shape of y_train_encoded:", y_train_encoded.shape)
print("Shape of y_pred_en:", y_pred_en.shape)

NameError: name 'y_train_encoded' is not defined

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X_imputed, Y, test_size=0.3, random_state=10)

In [ ]:
accuracy_score(y_train_encoded, y_pred_en)

In [ ]:
tree.plot_tree(clf_entropy)